# Data integration of district level data #
[NCES data](https://nces.ed.gov/ccd/elsi/tableGenerator.aspx), [STAAR data](https://tea.texas.gov/student-assessment/testing/staar/staar-aggregate-data), [Covid data](https://dshs.texas.gov/coronavirus/schools/texas-education-agency/), [ADA data](https://tea.texas.gov/finance-and-grants/state-funding/state-funding-reports-and-data/average-daily-attendance-and-wealth-per-average-daily-attendance), [LAUS data](https://www.bls.gov/lau/#cntyaa),  [Census Block Group 2010 TX](https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/census-block-group-2010-tx/), [ESSER data](https://tea.texas.gov/finance-and-grants/grants/grants-administration/applying-for-a-grant/entitlements)

This notebook is following these steps:

1. Integrating cleaned data from 7 different sources:
    - [NCES](https://nces.ed.gov/ccd/elsi/tableGenerator.aspx): DATA_NCES_DISTRICT.csv
    - [STAAR](https://tea.texas.gov/student-assessment/testing/staar/staar-aggregate-data): 
        - DATA_STAAR_DISTRICT_2019.csv
        - DATA_STAAR_DISTRICT_2021.csv
    - [Covid](https://dshs.texas.gov/coronavirus/schools/texas-education-agency/): 
        - DATA_COVID_DISTRICT.csv
        - DATA_COVID_COUNTY.csv
    - [LAUS(Local Area Unemployment Statistics)](https://www.bls.gov/lau/#cntyaa):
        - DATA_LAUS_COUNTY.csv
    - [ADA](https://tea.texas.gov/finance-and-grants/state-funding/state-funding-reports-and-data/average-daily-attendance-and-wealth-per-average-daily-attendance):
        - DATA_ADA_DISTRICT.csv
    - [Census Block Group 2010](https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/census-block-group-2010-tx/): 
        - DATA_Census_Block_Group_2010_TX_County.csv
    - [ESSER](https://tea.texas.gov/finance-and-grants/grants/grants-administration/applying-for-a-grant/entitlements)
        - DATA_ESSER_DISTRICT.csv     
      
2. Normalization: Normalizing numberical data into a percentage.  

3. Calculating Delta: Getting differences for the important variables having both values for 2018-2019 and 2020-2021  

4. Labeling: Creating 3 classes for Learning Loss  

5. Exporting: Generating 3 csv files for purposes:
    - DATA_Texas_District_v1.csv: raw integrated data with normalization, delta without missing value handling for EDA
    - DATA_Texas_District_v2.csv: dropping all missing values with normalization and delta values for Feature Selection and Baseline modeling
    - DATA_Texas_District_v3.csv: raw integrated data without normalization, delta. missing value handling for Gradient Boosting experiment

    
Note: List of variables is [here](https://git.txstate.edu/DataLab/data-NCES/blob/master/CHERR/EDA/DATA_Texas_District_List_of_Variables.csv).   

Note: All raw data was processed/cleaned from [here](https://git.txstate.edu/DataLab/data-NCES/blob/master/data).

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings("ignore")

## Reading [NCES data](https://nces.ed.gov/ccd/elsi/tableGenerator.aspx) ##

### NCES District ###

In [2]:
df_nces = pd.read_csv('data/DATA_NCES_DISTRICT.csv')
df_nces22 = pd.read_csv('DATA_NCES_DISTRICT22.csv')
print('Shape:', df_nces.shape)
df_nces.info()

FileNotFoundError: [Errno 2] No such file or directory: 'data/DATA_NCES_DISTRICT.csv'

In [ ]:
df_nces22

In [ ]:
df_nces

In [ ]:
df_nces = pd.merge(df_nces, df_nces22, how='left',on=['District #'])
df_nces

In [ ]:
temp=[c + ': ' + str(df_nces[c].isna().sum()) for c in df_nces if df_nces[c].isna().any()]
print("na rows:", len(temp))
temp

In [ ]:
df_nces.dropna().shape

## Reading [LAUS(Local Area Unemployment Statistics) Data ](https://www.bls.gov/lau/#cntyaa)  ##

In [ ]:
df_laus = pd.read_csv('data/DATA_LAUS_COUNTY.csv')
print('Shape:', df_laus.shape)
df_laus.info()

## Reading [STAAR data](https://tea.texas.gov/student-assessment/testing/staar/staar-aggregate-data) ##

### 2019 ###

In [ ]:
df_staar19 = pd.read_csv('data/DATA_STAAR_DISTRICT_2019.csv')
print('Shape:', df_staar19.shape)
print('Nunique District #:', df_staar19['District #'].nunique())
df_staar19.head()

In [ ]:
cols_staar19_all = ['r_all_rs_g3_2019' ,
                    'r_all_d_g3_2019' ,
                    'm_all_rs_g3_2019' ,
                    'm_all_d_g3_2019' ,
                    'r_all_rs_g4_2019' ,
                    'r_all_d_g4_2019' ,
                    'm_all_rs_g4_2019' ,
                    'm_all_d_g4_2019' ,
                    'r_all_rs_g5_2019' ,
                    'r_all_d_g5_2019' ,
                    'm_all_rs_g5_2019' ,
                    'm_all_d_g5_2019' ,
                    'r_all_rs_g6_2019' ,
                    'r_all_d_g6_2019' ,
                    'm_all_rs_g6_2019' ,
                    'm_all_d_g6_2019' ,
                    'r_all_rs_g7_2019' ,
                    'r_all_d_g7_2019' ,
                    'm_all_rs_g7_2019' ,
                    'm_all_d_g7_2019' ,
                    'r_all_rs_g8_2019' ,
                    'r_all_d_g8_2019' ,
                    'm_all_rs_g8_2019' ,
                    'm_all_d_g8_2019']

In [ ]:
df_staar19[cols_staar19_all].isna().sum()

### 2021 ###

In [3]:
df_staar21 = pd.read_csv('data/DATA_STAAR_DISTRICT_2021.csv')
print('Shape:', df_staar21.shape)
print('Nunique District #:', df_staar21['District #'].nunique())
df_staar21.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/DATA_STAAR_DISTRICT_2021.csv'

In [4]:
cols_staar21_all = ['r_all_rs_g3_2021',
                'r_all_d_g3_2021',
                'm_all_rs_g3_2021',
                'm_all_d_g3_2021',
                'r_all_rs_g4_2021',
                'r_all_d_g4_2021',
                'm_all_rs_g4_2021',
                'm_all_d_g4_2021',
                'r_all_rs_g5_2021',
                'r_all_d_g5_2021',
                'm_all_rs_g5_2021',
                'm_all_d_g5_2021',
                'r_all_rs_g6_2021',
                'r_all_d_g6_2021',
                'm_all_rs_g6_2021',
                'm_all_d_g6_2021',
                'r_all_rs_g7_2021',
                'r_all_d_g7_2021',
                'm_all_rs_g7_2021',
                'm_all_d_g7_2021',
                'r_all_rs_g8_2021',
                'r_all_d_g8_2021',
                'm_all_rs_g8_2021',
                'm_all_d_g8_2021']

In [5]:
df_staar21[cols_staar21_all].isna().sum()

NameError: name 'df_staar21' is not defined

### 2022 ###

In [ ]:
df_staar22 = pd.read_csv('data/DATA_STAAR_DISTRICT_2022.csv')
print('Shape:', df_staar22.shape)
print('Nunique District #:', df_staar22['District #'].nunique())
df_staar22.head()

In [ ]:
cols_staar22_all = ['r_all_rs_g3_2022',
                'r_all_d_g3_2022',
                'm_all_rs_g3_2022',
                'm_all_d_g3_2022',
                'r_all_rs_g4_2022',
                'r_all_d_g4_2022',
                'm_all_rs_g4_2022',
                'm_all_d_g4_2022',
                'r_all_rs_g5_2022',
                'r_all_d_g5_2022',
                'm_all_rs_g5_2022',
                'm_all_d_g5_2022',
                'r_all_rs_g6_2022',
                'r_all_d_g6_2022',
                'm_all_rs_g6_2022',
                'm_all_d_g6_2022',
                'r_all_rs_g7_2022',
                'r_all_d_g7_2022',
                'm_all_rs_g7_2022',
                'm_all_d_g7_2022',
                'r_all_rs_g8_2022',
                'r_all_d_g8_2022',
                'm_all_rs_g8_2022',
                'm_all_d_g8_2022']

In [ ]:
df_staar22[cols_staar22_all].isna().sum()

## Reading [Covid data](https://dshs.texas.gov/coronavirus/schools/texas-education-agency/) ##

### Covid District ###

In [ ]:
col_covid_dist=['District #',
                'District Enrollment 01/29/21',
                'Approx. District On-Campus Enrollment 01/29/21',
                'District Enrollment 10/30/20',
                'Approx. District On-Campus Enrollment 10/30/20',
                'District Enrollment 09/28/20',
                'Approx. District On-Campus Enrollment 09/28/20']
#                 'District Enrollment First Week',
#                 'Approx. District On-Campus Enrollment First Week']

In [ ]:
df_covid_dist = pd.read_csv('data/DATA_COVID_DISTRICT.csv', usecols=col_covid_dist)
print('Shape:', df_covid_dist.shape)
df_covid_dist.head()

In [ ]:
temp=[c + ': ' + str(df_covid_dist[c].isna().sum()) for c in df_covid_dist if df_covid_dist[c].isna().any()]
print("na rows:", len(temp))
temp

### Covid County ###

In [ ]:
df_nces['County #'].nunique()

In [ ]:
df_covid_county = pd.read_csv('data/DATA_COVID_COUNTY.csv')
print('Shape:', df_covid_county.shape)
df_covid_county.head()

In [ ]:
df_covid_county.drop(index=0, inplace=True)
df_covid_county.drop(columns='Unnamed: 0', axis=1, inplace=True)
print('Shape:', df_covid_county.shape)
df_covid_county.head()

In [ ]:
temp=[c + ': ' + str(df_covid_county[c].isna().sum()) for c in df_covid_county if df_covid_county[c].isna().any()]
print("na rows:", len(temp))
temp

In [ ]:
df_covid_county[~df_covid_county['County #'].isin(df_nces['County #'])]

## Reading ADA ##

**ada_county.csv is sum of ada_district.csv**

In [ ]:
cols_ada = ['District #',
        '2018-2019 ADA',
        '2020-2021 ADA',
        '2021-2022 ADA']

In [ ]:
df_ada = pd.read_csv('data/DATA_ADA_DISTRICT.csv', usecols = cols_ada)
df_ada.info()

In [ ]:
df_ada.dropna(inplace=True)
df_ada.info()

## Reading [Census Block Group 2010 TX](https://schoolsdata2-93b5c-tea-texas.opendata.arcgis.com/datasets/census-block-group-2010-tx/) data ##

In [ ]:
df_census = pd.read_csv('data/DATA_Census_Block_Group_2010_TX_County.csv')
print('Shape:', df_census.shape)
df_census.info()

## Reading [ESSER data](https://tea.texas.gov/finance-and-grants/grants/grants-administration/applying-for-a-grant/entitlements)

In [ ]:
df_esser = pd.read_csv('data/DATA_ESSER_DISTRICT.csv')
df_esser.info()

## Merge Files ##

1. Start from NCES `df_nces` merging with STAAR 2018-2019 `df_staar19'
1. merging LAUS `df_laus`
1. merging Census Block Group 2010 TX `df_census`
1. merging STAAR 2020-2021 `df_staar21`
1. merging Covid district `df_covid_dist`
1. merging Covid county `df_covid_county`
1. merging ADA `df_ada`
1. merging ESSER `df_esser`

In [ ]:
df_merge = pd.merge(df_nces, df_staar19, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_staar21, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_staar22, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_laus, how="left", on="County #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_census, how="left", on="County #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_covid_dist, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_covid_county, how="left", on="County #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_ada, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

In [ ]:
df_merge = pd.merge(df_merge, df_esser, how="left", on="District #")
temp=[c + ': ' + str(df_merge[c].isna().sum()) for c in df_merge if df_merge[c].isna().any()]
print("na rows: {}".format(len(temp)))
df_merge.info()

**STAAR na values:** check how many districts are not available to have 'Average Scale Score All Students' columns for all grades between 3 and 8.

In [ ]:
cols_staar19_all = ['District #',
                    'r_all_rs_g3_2019',
                    'm_all_rs_g3_2019',
                    'r_all_rs_g4_2019',
                    'm_all_rs_g4_2019',
                    'r_all_rs_g5_2019',
                    'm_all_rs_g5_2019',
                    'r_all_rs_g6_2019',
                    'm_all_rs_g6_2019',
                    'r_all_rs_g7_2019',
                    'm_all_rs_g7_2019',
                    'r_all_rs_g8_2019',
                    'm_all_rs_g8_2019']

In [ ]:
df_merge['count_na_staar_2019'] = df_merge[cols_staar19_all[1:]].apply(lambda x: x.isna().sum(), axis = 1)

In [ ]:
df_merge[df_merge['count_na_staar_2019']==len(cols_staar19_all)-1][cols_staar19_all].shape

In [ ]:
dist_remove_2019 = df_merge[df_merge['count_na_staar_2019']==len(cols_staar19_all)-1]['District #'].to_frame()
print('None of ''All students'' scores is available in', len(dist_remove_2019), 'districts for 2019')
dist_remove_2019

In [ ]:
cols_staar21_all = ['District #',
                    'r_all_rs_g3_2021',
                    'm_all_rs_g3_2021',
                    'r_all_rs_g4_2021',
                    'm_all_rs_g4_2021',
                    'r_all_rs_g5_2021',
                    'm_all_rs_g5_2021',
                    'r_all_rs_g6_2021',
                    'm_all_rs_g6_2021',
                    'r_all_rs_g7_2021',
                    'm_all_rs_g7_2021',
                    'r_all_rs_g8_2021',
                    'm_all_rs_g8_2021']

In [ ]:
df_merge['count_na_staar_2021'] = df_merge[cols_staar21_all[1:]].apply(lambda x: x.isna().sum(), axis = 1)

In [ ]:
dist_remove_2021 = df_merge[df_merge['count_na_staar_2021']==len(cols_staar21_all)-1]['District #'].to_frame()
print('None of ''All students'' scores is available in', len(dist_remove_2021), 'districts for 2021')

In [ ]:
cols_staar22_all = ['District #',
                    'r_all_rs_g3_2022',
                    'm_all_rs_g3_2022',
                    'r_all_rs_g4_2022',
                    'm_all_rs_g4_2022',
                    'r_all_rs_g5_2022',
                    'm_all_rs_g5_2022',
                    'r_all_rs_g6_2022',
                    'm_all_rs_g6_2022',
                    'r_all_rs_g7_2022',
                    'm_all_rs_g7_2022',
                    'r_all_rs_g8_2022',
                    'm_all_rs_g8_2022']

In [ ]:
df_merge['count_na_staar_2022'] = df_merge[cols_staar22_all[1:]].apply(lambda x: x.isna().sum(), axis = 1)

In [ ]:
dist_remove_2022 = df_merge[df_merge['count_na_staar_2022']==len(cols_staar22_all)-1]['District #'].to_frame()
print('None of ''All students'' scores is available in', len(dist_remove_2022), 'districts for 2022')

In [ ]:
dist_remove = pd.merge(dist_remove_2019, dist_remove_2021, how='outer', on='District #')

In [ ]:
dist_remove = pd.merge(dist_remove, dist_remove_2022, how='outer', on='District #')

In [ ]:
cols_removed = cols_staar19_all[1:] + cols_staar21_all[1:] + cols_staar22_all[1:]
print('Rows that are not availale for the variables below are removed:')
cols_removed

In [ ]:
dist_remove_index = df_merge[df_merge['District #'].isin(dist_remove['District #'])].index
print(dist_remove_index)
df_merge.drop(dist_remove_index, inplace=True)
df_merge.drop(columns=['count_na_staar_2019', 'count_na_staar_2021','count_na_staar_2022'], inplace=True)
df_merge.info()

In [ ]:
plt.rcParams['figure.figsize'] = (20,6)
sns.heatmap(df_merge.isna(), yticklabels = False, cbar = False , cmap = 'viridis')
plt.title("Missing null values")
plt.show()

In [ ]:
df=df_merge.copy()
cols_norm=[]

## Normalization:
Normalizing numberical data into a percentage.

- Total Operational Public Schools
- School-wide Title 1
- Title 1 Eligible School

df['Total Operational Public Schools 2021-2022'] = df['Total Operational Public Schools 2020-2021']
df['School-wide Title I 2021-2022'] = df['School-wide Title I 2020-2021']
df['Title I Eligible School 2021-2022'] = df['Title I Eligible School 2020-2021']
df['Total Public Schools 2021-2022'] = df['Total Public Schools 2020-2021']
df['Total Students 2021-2022'] = df['Total Students 2020-2021']

In [ ]:
cols_sch = ['Total Operational Public Schools 2021-2022',
            'Total Operational Public Schools 2020-2021',
            'Total Operational Public Schools 2018-2019',
            'School-wide Title I 2021-2022',
            'School-wide Title I 2020-2021',
            'School-wide Title I 2018-2019',
            'Title I Eligible School 2021-2022',
            'Title I Eligible School 2020-2021',
            'Title I Eligible School 2018-2019']

for col in cols_sch:
    if col[-4:] == '2019': 
        df['% '+col] = df[col] / df['Total Public Schools 2018-2019']
        cols_norm.append('% '+col)
    elif col[-4:] == '2022':
        df['% '+col] = df[col] / df['Total Public Schools 2021-2022']
        cols_norm.append('% '+col)
    else: 
        df['% '+col] = df[col] / df['Total Public Schools 2020-2021']
        cols_norm.append('% '+col)

- Teachers & Staff: FTE Teachers, Total Staff
- Lunch Program: Free Lunch, Reduced-price Lunch
- Race/Ethnicity: Asian, Hispanic, Black, White
- Grade group(level): Grades 1-8, Grades 9-12, Prekindergarten, Kindergarten 
- Grade (each): Grade 1 to Grade 12
- Average Daily Attendance(ADA)

In [ ]:
df['Free or Reduced Lunch 2020-2021'] = df['Free Lunch 2020-2021'] + df['Reduced-price Lunch 2020-2021']
df['Free or Reduced Lunch 2018-2019'] = df['Free Lunch 2018-2019'] + df['Reduced-price Lunch 2018-2019']

In [ ]:
cols_old = ['FTE Teachers 2020-2021',
            'FTE Teachers 2018-2019',
            'Total Staff 2020-2021',
            'Total Staff 2018-2019',
           # 'Free Lunch 2020-2021',
           # 'Free Lunch 2018-2019',
           # 'Reduced-price Lunch 2020-2021',
           # 'Reduced-price Lunch 2018-2019',
            'Free or Reduced Lunch 2020-2021',
            'Free or Reduced Lunch 2018-2019',
            'Asian or Asian/Pacific Islander Students 2020-2021',
            'Asian or Asian/Pacific Islander Students 2018-2019',
            'Hispanic Students 2020-2021',
            'Hispanic Students 2018-2019',
            'Black or African American Students 2020-2021',
            'Black or African American Students 2018-2019',
            'White Students 2020-2021',
            'White Students 2018-2019',
            'Grades 1-8 2020-2021',
            'Grades 1-8 2018-2019',
            'Grades 9-12 2020-2021',
            'Grades 9-12 2018-2019',
            'Prek 2020-2021',
            'Prek 2018-2019',
            'K 2020-2021',
            'K 2018-2019',
            'Grade 1 2020-2021',
            'Grade 1 2018-2019',
            'Grade 2 2020-2021',
            'Grade 2 2018-2019',
            'Grade 3 2020-2021',
            'Grade 3 2018-2019',
            'Grade 4 2020-2021',
            'Grade 4 2018-2019',
            'Grade 5 2020-2021',
            'Grade 5 2018-2019',
            'Grade 6 2020-2021',
            'Grade 6 2018-2019',
            'Grade 7 2020-2021',
            'Grade 7 2018-2019',
            'Grade 8 2020-2021',
            'Grade 8 2018-2019',
            'Grade 9 2020-2021',
            'Grade 9 2018-2019',
            'Grade 10 2020-2021',
            'Grade 10 2018-2019',
            'Grade 11 2020-2021',
            'Grade 11 2018-2019',
            'Grade 12 2020-2021',
            'Grade 12 2018-2019',
            '2018-2019 ADA',
            '2020-2021 ADA',
            'FTE Teachers 2021-2022',
            'Total Staff 2021-2022',
            'Free or Reduced Lunch 2021-2022',
            'Asian or Asian/Pacific Islander Students 2021-2022',
            'Hispanic Students 2021-2022',
            'Black or African American Students 2021-2022',
            'White Students 2021-2022',
            'Grades 1-8 2021-2022',
            'Grades 9-12 2021-2022',
            'Prek 2021-2022',
            'K 2021-2022',
            'Grade 1 2021-2022',
            'Grade 2 2021-2022',
            'Grade 3 2021-2022',
            'Grade 4 2021-2022',
            'Grade 5 2021-2022',
            'Grade 6 2021-2022',
            'Grade 7 2021-2022',
            'Grade 8 2021-2022',
            'Grade 9 2021-2022',
            'Grade 10 2021-2022',
            'Grade 11 2021-2022',
            'Grade 12 2021-2022',
            '2021-2022 ADA']

cols_new = ['Teachers:Students 2020-2021',
            'Teachers:Students 2018-2019',
            'Staff:Students 2020-2021',
            'Staff:Students 2018-2019',
           # '% Free Lunch 2020-2021',
           # '% Free Lunch 2018-2019',
           # '% Reduced-price Lunch 2020-2021',
           # '% Reduced-price Lunch 2018-2019',
            '% Free or Reduced-price Lunch 2020-2021',
            '% Free or Reduced-price Lunch 2018-2019',
            '% Asian or Asian/Pacific Islander Students 2020-2021',
            '% Asian or Asian/Pacific Islander Students 2018-2019',
            '% Hispanic Students 2020-2021',
            '% Hispanic Students 2018-2019',
            '% Black or African American Students 2020-2021',
            '% Black or African American Students 2018-2019',
            '% White Students 2020-2021',
            '% White Students 2018-2019',
            '% Grades 1-8 2020-2021',
            '% Grades 1-8 2018-2019',
            '% Grades 9-12 2020-2021',
            '% Grades 9-12 2018-2019',
            '% Prek 2020-2021',
            '% Prek 2018-2019',
            '% K 2020-2021',
            '% K 2018-2019',
            '% Grade 1 2020-2021',
            '% Grade 1 2018-2019',
            '% Grade 2 2020-2021',
            '% Grade 2 2018-2019',
            '% Grade 3 2020-2021',
            '% Grade 3 2018-2019',
            '% Grade 4 2020-2021',
            '% Grade 4 2018-2019',
            '% Grade 5 2020-2021',
            '% Grade 5 2018-2019',
            '% Grade 6 2020-2021',
            '% Grade 6 2018-2019',
            '% Grade 7 2020-2021',
            '% Grade 7 2018-2019',
            '% Grade 8 2020-2021',
            '% Grade 8 2018-2019',
            '% Grade 9 2020-2021',
            '% Grade 9 2018-2019',
            '% Grade 10 2020-2021',
            '% Grade 10 2018-2019',
            '% Grade 11 2020-2021',
            '% Grade 11 2018-2019',
            '% Grade 12 2020-2021',
            '% Grade 12 2018-2019',
            '% ADA 2018-2019',
            '% ADA 2020-2021',
            'Teachers:Students 2021-2022',
            'Staff:Students 2021-2022',
            'Staff:Students 2018-2019',
            '% Free or Reduced-price Lunch 2021-2022',
            '% Asian or Asian/Pacific Islander Students 2021-2022',
            '% Hispanic Students 2021-2022',
            '% Black or African American Students 2021-2022',
            '% White Students 2021-2022',
            '% Grades 1-8 2021-2022',
            '% Grades 9-12 2021-2022',
            '% Prek 2021-2022',
            '% K 2021-2022',
            '% Grade 1 2021-2022',
            '% Grade 2 2021-2022',
            '% Grade 3 2021-2022',
            '% Grade 4 2021-2022',
            '% Grade 5 2021-2022',
            '% Grade 6 2021-2022',
            '% Grade 7 2021-2022',
            '% Grade 8 2021-2022',
            '% Grade 9 2021-2022',
            '% Grade 10 2021-2022',
            '% Grade 11 2021-2022',
            '% Grade 12 2021-2022',
            '% ADA 2021-2022'
           ]


for old, new in zip(cols_old, cols_new):
    if old[-4:] == '2019': 
        df[new] = df[old] / df['Total Students 2018-2019']
        cols_norm.append(new)
    elif old[-4:] == '2022':
        df[new] = df[old] / df['Total Students 2021-2022']
    else: 
        df[new] = df[old] / df['Total Students 2020-2021']
        cols_norm.append(new)


for old, new in zip(temp_col, cols_22):
        df[new] = df[old] / df['Total Students 2020-2021']
        cols_norm.append(new)
for old, new in zip(temp_col, temp_22):
        df[new] = df[old]

temp_col = ['FTE Teachers 2020-2021',
            'Total Staff 2020-2021',
            'Free Lunch 2020-2021',
            'Reduced-price Lunch 2020-2021',
            'Asian or Asian/Pacific Islander Students 2020-2021',
            'Hispanic Students 2020-2021',
            'Black or African American Students 2020-2021',
            'White Students 2020-2021',
            'Grades 1-8 2020-2021',
            'Grades 9-12 2020-2021',
            'Prek 2020-2021',
            'K 2020-2021',
            'Grade 1 2020-2021',
            'Grade 2 2020-2021',
            'Grade 3 2020-2021',
            'Grade 4 2020-2021',
            'Grade 5 2020-2021',
            'Grade 6 2020-2021',
            'Grade 7 2020-2021',
            'Grade 8 2020-2021',
            'Grade 9 2020-2021',
            'Grade 10 2020-2021',
            'Grade 11 2020-2021',
            'Grade 12 2020-2021',
            '2021-2022 ADA']

temp_22 = ['FTE Teachers 2021-2022',
            'Total Staff 2021-2022',
            'Free Lunch 2021-2022',
            'Reduced-price Lunch 2021-2022',
            'Asian or Asian/Pacific Islander Students 2021-2022',
            'Hispanic Students 2021-2022',
            'Black or African American Students 2021-2022',
            'White Students 2021-2022',
            'Grades 1-8 2021-2022',
            'Grades 9-12 2021-2022',
            'Prek 2021-2022',
            'K 2021-2022',
            'Grade 1 2021-2022',
            'Grade 2 2021-2022',
            'Grade 3 2021-2022',
            'Grade 4 2021-2022',
            'Grade 5 2021-2022',
            'Grade 6 2021-2022',
            'Grade 7 2021-2022',
            'Grade 8 2021-2022',
            'Grade 9 2021-2022',
            'Grade 10 2021-2022',
            'Grade 11 2021-2022',
            'Grade 12 2021-2022',
            '2021-2022 ADA']

cols_22 = ['Teachers:Students 2021-2022',
           'Staff:Students 2021-2022',
           '% Free Lunch 2021-2022',
           '% Reduced-price Lunch 2021-2022',
           '% Asian or Asian/Pacific Islander Students 2021-2022',
           '% Hispanic Students 2021-2022',
           '% Black or African American Students 2021-2022',
           '% White Students 2021-2022',
           '% Grades 1-8 2021-2022',
           '% Grades 9-12 2021-2022',
           '% Prek 2021-2022',
           '% K 2021-2022',
           '% Grade 1 2021-2022',
           '% Grade 2 2021-2022',
           '% Grade 3 2021-2022',
           '% Grade 4 2021-2022',
           '% Grade 5 2021-2022',
           '% Grade 6 2021-2022',
           '% Grade 7 2021-2022',
           '% Grade 8 2021-2022',
           '% Grade 9 2021-2022',
           '% Grade 10 2021-2022',
           '% Grade 11 2021-2022',
           '% Grade 12 2021-2022',
           '% ADA 2021-2022']


- Percent of total students that took the STAAR Reading and Math tests per grade

In [ ]:
cols_math = ['m_all_d_g3_2019',
            'm_all_d_g4_2019',
            'm_all_d_g5_2019',
            'm_all_d_g6_2019',
            'm_all_d_g7_2019',
            'm_all_d_g8_2019',
            'm_all_d_g3_2021',
            'm_all_d_g4_2021',
            'm_all_d_g5_2021',
            'm_all_d_g6_2021',
            'm_all_d_g7_2021',
            'm_all_d_g8_2021',
            'm_all_d_g3_2022',
            'm_all_d_g4_2022',
            'm_all_d_g5_2022',
            'm_all_d_g6_2022',
            'm_all_d_g7_2022',
            'm_all_d_g8_2022']

cols_reading = ['r_all_d_g3_2019',
            'r_all_d_g4_2019',
            'r_all_d_g5_2019',
            'r_all_d_g6_2019',
            'r_all_d_g7_2019',
            'r_all_d_g8_2019',
            'r_all_d_g3_2021',
            'r_all_d_g4_2021',
            'r_all_d_g5_2021',
            'r_all_d_g6_2021',
            'r_all_d_g7_2021',
            'r_all_d_g8_2021',
            'r_all_d_g3_2022',
            'r_all_d_g4_2022',
            'r_all_d_g5_2022',
            'r_all_d_g6_2022',
            'r_all_d_g7_2022',
            'r_all_d_g8_2022']

cols_grade = ['Grade 3 2018-2019',
            'Grade 4 2018-2019',
            'Grade 5 2018-2019',
            'Grade 6 2018-2019',
            'Grade 7 2018-2019',
            'Grade 8 2018-2019',
            'Grade 3 2020-2021',
            'Grade 4 2020-2021',
            'Grade 5 2020-2021',
            'Grade 6 2020-2021',
            'Grade 7 2020-2021',
            'Grade 8 2020-2021',
            'Grade 3 2021-2022',
            'Grade 4 2021-2022',
            'Grade 5 2021-2022',
            'Grade 6 2021-2022',
            'Grade 7 2021-2022',
            'Grade 8 2021-2022']

for math, reading, grade in zip(cols_math, cols_reading, cols_grade):
    df['% Students Tested Math - ' + grade] = df[[math, grade]].apply(lambda x: np.nan if x[grade]==0
                                                                      else x[math] / x[grade], axis=1)
    df['% Students Tested Reading - ' + grade] = df[[reading, grade]].apply(lambda x: np.nan if x[grade]==0
                                                                      else x[reading] / x[grade], axis=1)
    cols_norm.append('% Students Tested Math - ' + grade)
    cols_norm.append('% Students Tested Reading - ' + grade)

- Average Math and Reading score as a ratio to the 100th percentile score in each grade

In [ ]:
cols_math =['m_all_rs',
            'm_ethh_rs',
            'm_etha_rs',
            'm_ethb_rs',
            'm_ethw_rs',
            'm_ecoy_rs',
            'm_eco1_rs',
            'm_ti1y_rs',
            'm_spey_rs']

cols_reading = ['r_all_rs',
                'r_ethh_rs',
                'r_etha_rs',
                'r_ethb_rs',
                'r_ethw_rs',
                'r_ecoy_rs',
                'r_eco1_rs',
                'r_ti1y_rs',
                'r_spey_rs']

cols_group = ['All',
            'Hispanic',
            'Asian',
            'Black',
            'White',
            'Poverty',
            'Free Lunch',
            'T1',
            'Special Ed']

for year in ['2018-2019', '2020-2021','2021-2022']:
    for grade in ['3', '4', '5', '6', '7', '8']:
        max_math = df['m_all_rs_g' + grade + '_' + year[-4:]].quantile(1)
        max_reading = df['r_all_rs_g' + grade + '_' + year[-4:]].quantile(1)     

        for math, reading, group in zip(cols_math, cols_reading, cols_group):
            math_old = math + '_g' + grade + '_' + year[-4:]
            reading_old = reading + '_g' + grade + '_' + year[-4:]
            math_new = 'Average Score Math ' + group + ' Grade ' + grade + ' ' + year
            reading_new = 'Average Score Reading ' + group + ' Grade ' + grade + ' ' + year
            df[math_new] = df[math_old].apply(lambda x: x/max_math)
            df[reading_new] = df[reading_old].apply(lambda x: x/max_reading)
            cols_norm.append(math_new)
            cols_norm.append(reading_new)

- Average daily attendance as a percent of Total Students per district

In [ ]:
df['% ADA 2018-2019'] = df['2018-2019 ADA'] / df['Total Students 2018-2019']
df['% ADA 2020-2021'] = df['2020-2021 ADA'] / df['Total Students 2020-2021']
df['% ADA 2021-2022'] = df['2021-2022 ADA'] / df['Total Students 2021-2022']
cols_norm.append('% ADA 2018-2019')
cols_norm.append('% ADA 2020-2021')
cols_norm.append('% ADA 2021-2022')

- Confirmed Cases and Confirmed Deathes as a percent of County Population.

In [ ]:
df['% County Infected 09/28/20'] = df.apply(lambda x: x['Confirmed Cases 09/28/20'] / x['County Population'], axis=1)
df['% County Deaths 09/28/20'] = df.apply(lambda x: x['Confirmed Deaths 09/28/20'] / x['County Population'], axis=1)

df['% County Infected 10/30/20'] = df.apply(lambda x: x['Confirmed Cases 10/30/20'] / x['County Population'], axis=1)
df['% County Deaths 10/30/20'] = df.apply(lambda x: x['Confirmed Deaths 10/30/20'] / x['County Population'], axis=1)

df['% County Infected 01/29/21'] = df.apply(lambda x: x['Confirmed Cases 01/29/21'] / x['County Population'], axis=1)
df['% County Deaths 01/29/21'] = df.apply(lambda x: x['Confirmed Deaths 01/29/21'] / x['County Population'], axis=1)

df['% County Infected 07/28/2021'] = df.apply(lambda x: x['Confirmed Cases 07/28/2021'] / x['County Population'], axis=1)
df['% County Deaths 07/28/2021'] = df.apply(lambda x: x['Confirmed Deaths 07/28/2021'] / x['County Population'], axis=1)

df['% County Infected 01/29/2022'] = df.apply(lambda x: x['Confirmed Cases 01/29/2022'] / x['County Population'], axis=1)
df['% County Deaths 01/29/2022'] = df.apply(lambda x: x['Confirmed Deaths 01/29/2022'] / x['County Population'], axis=1)

df['% County Infected 07/29/2022'] = df.apply(lambda x: x['Confirmed Cases 07/29/2022'] / x['County Population'], axis=1)
df['% County Deaths 07/29/2022'] = df.apply(lambda x: x['Confirmed Deaths 07/29/2022'] / x['County Population'], axis=1)

df['% County Infected 10/30/2022'] = df.apply(lambda x: x['Confirmed Cases 10/30/2022'] / x['County Population'], axis=1)
df['% County Deaths 10/30/2022'] = df.apply(lambda x: x['Confirmed Deaths 10/30/2022'] / x['County Population'], axis=1)


cols_norm.append('% County Infected 09/28/20')
cols_norm.append('% County Deaths 09/28/20')
cols_norm.append('% County Infected 10/30/20')
cols_norm.append('% County Deaths 10/30/20')
cols_norm.append('% County Infected 01/29/21')
cols_norm.append('% County Deaths 01/29/21')
cols_norm.append('% County Infected 07/28/21')
cols_norm.append('% County Deaths 07/28/21')
cols_norm.append('% County Deaths 01/29/2022')
cols_norm.append('% County Infected 01/29/2022')
cols_norm.append('% County Deaths 07/29/2022')
cols_norm.append('% County Infected 07/29/2022')
cols_norm.append('% County Deaths 10/30/2022')
cols_norm.append('% County Infected 10/30/2022')

- Percent of the students on campus for each date 09/28/20, 10/30/20, and 01/29/21.

In [ ]:
df['% On Campus 09/28/20'] = df.apply(lambda x: x['Approx. District On-Campus Enrollment 09/28/20'] / x['District Enrollment 09/28/20'], axis=1)
df['% On Campus 10/30/20'] = df.apply(lambda x: x['Approx. District On-Campus Enrollment 10/30/20'] / x['District Enrollment 10/30/20'], axis=1)
df['% On Campus 01/29/21'] = df.apply(lambda x: x['Approx. District On-Campus Enrollment 01/29/21'] / x['District Enrollment 01/29/21'], axis=1)

cols_norm.append('% On Campus 09/28/20')
cols_norm.append('% On Campus 10/30/20')
cols_norm.append('% On Campus 01/29/21')

- Race/Ethnicity and Age groups as a percent of County Population in 2010 (`Population 2010`)

In [ ]:
cols_pop = ['White Pop 10',
            'Black Pop 10',
            'Asian Pop 10',
            'Hispanic Pop 10',
            'Male Pop 10',
            'Female Pop 10',
            'Age 0-4 Pop 10',
            'Age 5-9 Pop 10',
            'Age 10-14 Pop 10',
            'Age 15-19 Pop 10',
            'Age 20-24 Pop 10',
            'Age 25-34 Pop 10',
            'Age 35-44 Pop 10',
            'Age 45-54 Pop 10',
            'Age 55-64 Pop 10',
            'Age 65-74 Pop 10',
            'Age 75-84 Pop 10',
            'Age 85-Up Pop 10']

for c in cols_pop:
    df['% '+c] = df.apply(lambda x: x[c] / x['Population 10'], axis=1)
    cols_norm.append('% '+c)

- Number of different types of Households as a percent of Total Number of Households in 2010 (`# of Households 10`)

In [ ]:
cols_hh = ['HH 1 Male 10',
            'HH 1 Female 10',
            'HH Married-Child 10',
            'HH Married-noChild 10',
            'HH Male-Child 10',
            'HH Female-Child 10']

for c in cols_hh:
    df['% '+c] = df.apply(lambda x: x[c] / x['# of Households 10'], axis=1)
    cols_norm.append('% '+c)

- Number of different types of Housing Units as a percent of Total Number of Housing Units in 2010 (`# of Housing Units 10`)

In [ ]:
cols_house = ['Housing Vacant 10',
            'Housing Owner Occup 10',
            'Housing Renter Occup 10']

for c in cols_house:
    df['% '+c] = df.apply(lambda x: x[c] / x['# of Housing Units 10'], axis=1)
    cols_norm.append('% '+c)

## Calculating Delta: 
Getting differences for the important variables having both values for 2018-2019 and 2020-2021 and 2021-2022

In [ ]:
cols_diff = ['Total Public Schools',
            'Total Students',
            '% Total Operational Public Schools',
            '% School-wide Title I',
            '% Title I Eligible School',
            'Teachers:Students',
            'Staff:Students',
            '% Free or Reduced-price Lunch',
            '% Asian or Asian/Pacific Islander Students',
            '% Hispanic Students',
            '% Black or African American Students',
            '% White Students',
            '% Grades 1-8',
            '% Grades 9-12',
            '% Prek',
            '% K',
            '% ADA',
            '% Students Tested Math - Grade 3',
            '% Students Tested Reading - Grade 3',
            '% Students Tested Math - Grade 4',
            '% Students Tested Reading - Grade 4',
            '% Students Tested Math - Grade 5',
            '% Students Tested Reading - Grade 5',
            '% Students Tested Math - Grade 6',
            '% Students Tested Reading - Grade 6',
            '% Students Tested Math - Grade 7',
            '% Students Tested Reading - Grade 7',
            '% Students Tested Math - Grade 8',
            '% Students Tested Reading - Grade 8',
            '% Grade 1',
            '% Grade 2',
            '% Grade 3',
            '% Grade 4',
            '% Grade 5',
            '% Grade 6',
            '% Grade 7',
            '% Grade 8',
            '% Grade 9',
            '% Grade 10',
            '% Grade 11',
            '% Grade 12']
cols_delta =[]

In [ ]:
for var in cols_diff:
    df[var + ' Diff 2019-2021'] = df.apply(lambda x: 0 if x[[var + ' 2018-2019', var + ' 2020-2021']].sum() == 0
                                 else np.nan if x[var + ' 2018-2019'] == 0 
                                 else np.nan if x[[var + ' 2018-2019', var + ' 2020-2021']].isna().any()
                                 else (x[var + ' 2020-2021'] - x[var + ' 2018-2019']) / x[var + ' 2018-2019'], axis=1)
    cols_delta.append(var + ' Diff 2019-2021')

In [ ]:
for var in cols_diff:
    df[var + ' Diff 2021-2022'] = df.apply(lambda x: 0 if x[[var + ' 2020-2021', var + ' 2021-2022']].sum() == 0
                                 else np.nan if x[var + ' 2020-2021'] == 0 
                                 else np.nan if x[[var + ' 2020-2021', var + ' 2021-2022']].isna().any()
                                 else (x[var + ' 2021-2022'] - x[var + ' 2020-2021']) / x[var + ' 2020-2021'], axis=1)
    cols_delta.append(var + ' Diff 2021-2022')

In [ ]:
for var in cols_diff:
    df[var + ' Diff 2019-2022'] = df.apply(lambda x: 0 if x[[var + ' 2018-2019', var + ' 2021-2022']].sum() == 0
                                 else np.nan if x[var + ' 2018-2019'] == 0 
                                 else np.nan if x[[var + ' 2018-2019', var + ' 2021-2022']].isna().any()
                                 else (x[var + ' 2021-2022'] - x[var + ' 2018-2019']) / x[var + ' 2018-2019'], axis=1)
    cols_delta.append(var + ' Diff 2019-2022')

## Labeling ##   
Creating 3 classes for Learning Loss  
- Step 1: Learning Loss is calculated by averaging Average Score of all grades for Math and Reading separately and substracting the 2021 average from 2019 average.

In [ ]:
cols_math_2019 = ['Average Score Math All Grade 3 2018-2019',
                'Average Score Math All Grade 4 2018-2019',
                'Average Score Math All Grade 5 2018-2019',
                'Average Score Math All Grade 6 2018-2019',
                'Average Score Math All Grade 7 2018-2019',
                'Average Score Math All Grade 8 2018-2019']

cols_reading_2019 = ['Average Score Reading All Grade 3 2018-2019',
                'Average Score Reading All Grade 4 2018-2019',
                'Average Score Reading All Grade 5 2018-2019',
                'Average Score Reading All Grade 6 2018-2019',
                'Average Score Reading All Grade 7 2018-2019',
                'Average Score Reading All Grade 8 2018-2019']

cols_math_2021 = ['Average Score Math All Grade 3 2020-2021',
                'Average Score Math All Grade 4 2020-2021',
                'Average Score Math All Grade 5 2020-2021',
                'Average Score Math All Grade 6 2020-2021',
                'Average Score Math All Grade 7 2020-2021',
                'Average Score Math All Grade 8 2020-2021']

cols_reading_2021 = ['Average Score Reading All Grade 3 2020-2021',
                'Average Score Reading All Grade 4 2020-2021',
                'Average Score Reading All Grade 5 2020-2021',
                'Average Score Reading All Grade 6 2020-2021',
                'Average Score Reading All Grade 7 2020-2021',
                'Average Score Reading All Grade 8 2020-2021']

cols_math_2022 = ['Average Score Math All Grade 3 2021-2022',
                'Average Score Math All Grade 4 2021-2022',
                'Average Score Math All Grade 5 2021-2022',
                'Average Score Math All Grade 6 2021-2022',
                'Average Score Math All Grade 7 2021-2022',
                'Average Score Math All Grade 8 2021-2022']

cols_reading_2022 = ['Average Score Reading All Grade 3 2021-2022',
                'Average Score Reading All Grade 4 2021-2022',
                'Average Score Reading All Grade 5 2021-2022',
                'Average Score Reading All Grade 6 2021-2022',
                'Average Score Reading All Grade 7 2021-2022',
                'Average Score Reading All Grade 8 2021-2022']

df['AVG_Math_2019'] = df[cols_math_2019].apply(lambda x: x.sum() / len(cols_math_2019), axis=1)
df['AVG_Reading_2019'] = df[cols_reading_2019].apply(lambda x: x.sum() / len(cols_reading_2019), axis=1)
df['AVG_Math_2021'] = df[cols_math_2021].apply(lambda x: x.sum() / len(cols_math_2021), axis=1)
df['AVG_Reading_2021'] = df[cols_reading_2021].apply(lambda x: x.sum() / len(cols_reading_2021), axis=1)
df['AVG_Math_2022'] = df[cols_math_2022].apply(lambda x: x.sum() / len(cols_math_2022), axis=1)
df['AVG_Reading_2022'] = df[cols_reading_2022].apply(lambda x: x.sum() / len(cols_reading_2022), axis=1)

Get Percent Loss Different from 2019 to 2021 for the perspective of 2019. % LOSS = (AVG 2021 - AVG 2019)/AVG 2019

if '% Loss' Difference > 0, 2021 Learning is better than 2019, that is there is Learning Gain   
if '% Loss' Difference < 0, 2021 Learning is worse than 2019, that is there is Learning Loss

In [ ]:
df['% LOSS Math 2019 to 2021'] = (df['AVG_Math_2021'] - df['AVG_Math_2019']) / df['AVG_Math_2019']
df['% LOSS Reading 2019 to 2021'] = (df['AVG_Reading_2021'] - df['AVG_Reading_2019']) / df['AVG_Reading_2019']
df['% LOSS All 2019 to 2021'] = df[['% LOSS Math 2019 to 2021', '% LOSS Reading 2019 to 2021']].apply(lambda x: x.mean(), axis=1)

Get Percent Loss Different from 2020 to 2022 for the perspective of 2020. % LOSS = (AVG 2022 - AVG 2020)/AVG 2020

if '% Loss' Difference > 0, 2022 Learning is better than 2020, that is there is Learning Gain   
if '% Loss' Difference < 0, 2022 Learning is worse than 2020, that is there is Learning Loss

In [ ]:
df['% LOSS Math 2021 to 2022'] = (df['AVG_Math_2022'] - df['AVG_Math_2021']) / df['AVG_Math_2021']
df['% LOSS Reading 2021 to 2022'] = (df['AVG_Reading_2022'] - df['AVG_Reading_2021']) / df['AVG_Reading_2021']
df['% LOSS All 2021 to 2022'] = df[['% LOSS Math 2021 to 2022', '% LOSS Reading 2021 to 2022']].apply(lambda x: x.mean(), axis=1)

In [ ]:
df['% LOSS Math 2019 to 2022'] = (df['AVG_Math_2022'] - df['AVG_Math_2019']) / df['AVG_Math_2019']
df['% LOSS Reading 2019 to 2022'] = (df['AVG_Reading_2022'] - df['AVG_Reading_2019']) / df['AVG_Reading_2019']
df['% LOSS All 2019 to 2022'] = df[['% LOSS Math 2019 to 2022', '% LOSS Reading 2019 to 2022']].apply(lambda x: x.mean(), axis=1)

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
sns.set(font_scale=1.2)

g = sns.histplot(data=df, x='% LOSS Math 2019 to 2021', shrink=0.8, color='#8de4a1', label='Math', element="step", stat="count")
g = sns.histplot(data=df, x='% LOSS Reading 2019 to 2021', shrink=0.8, color='#ed7953', label='Reading', element="step", stat="count")
g.set(xlabel='% Loss', ylabel='Number of Districts', title='% Loss - Subject 2019-2021')
g.set_xlim(-.75, .75)
plt.legend(loc='upper right', fontsize=20)
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(8,5)})
sns.set(font_scale=1.2)

g = sns.histplot(data=df, x='% LOSS Math 2021 to 2022', shrink=0.8, color='#8de4a1', label='Math', element="step", stat="count")
g = sns.histplot(data=df, x='% LOSS Reading 2021 to 2022', shrink=0.8, color='#ed7953', label='Reading', element="step", stat="count")
g.set(xlabel='% Loss', ylabel='Number of Districts', title='% Loss - Subject 2021-2022')
g.set_xlim(-.75, .75)
plt.legend(loc='upper right', fontsize=20)
plt.show()

In [ ]:
cols = ['% LOSS Math 2019 to 2021', '% LOSS Reading 2019 to 2021', '% LOSS All 2019 to 2021']
df[cols].describe()

In [ ]:
cols = ['% LOSS Math 2021 to 2022', '% LOSS Reading 2021 to 2022', '% LOSS All 2021 to 2022']
df[cols].describe()

- Step 2: Decide threshold for labelings using `% LOSS Math` and `% LOSS Reading` separately.
    - 0 ***(Expected)***:  25th percentile <= x <= 75th percentile
    - 1 ***(Gain)***: 75th percentile < x
    - -1 ***(Loss)***:  x < 25th percentile

In [ ]:
mean_math = df['% LOSS Math 2019 to 2021'].mean()
std_math = df['% LOSS Math 2019 to 2021'].std()
q_25_math = df['% LOSS Math 2019 to 2021'].quantile(0.25)
q_75_math = df['% LOSS Math 2019 to 2021'].quantile(0.75)

mean_reading = df['% LOSS Reading 2019 to 2021'].mean()
std_reading = df['% LOSS Reading 2019 to 2021'].std()
q_25_reading = df['% LOSS Reading 2019 to 2021'].quantile(0.25)
q_75_reading = df['% LOSS Reading 2019 to 2021'].quantile(0.75)

mean_all = df['% LOSS All 2019 to 2021'].mean()
std_all = df['% LOSS All 2019 to 2021'].std()
q_25_all = df['% LOSS All 2019 to 2021'].quantile(0.25)
q_75_all = df['% LOSS All 2019 to 2021'].quantile(0.75)

sig = 1
sig_pos_math = mean_math + (sig * std_math)
sig_neg_math = mean_math - (sig * std_math)
sig_pos_reading = mean_reading + (sig * std_reading)
sig_neg_reading = mean_reading - (sig * std_reading)
sig_pos_all = mean_all + (sig * std_all)
sig_neg_all = mean_all - (sig * std_all)

df['Label_Math (19-21)'] = df['% LOSS Math 2019 to 2021'].apply(lambda x: 0 if ((x >= q_25_math) & (x <= q_75_math))
                                   else 1 if (x > q_75_math)
                                   else -1 if (x < q_25_math)
                                   else -100)   

df['Label_Reading (19-21)'] = df['% LOSS Reading 2019 to 2021'].apply(lambda x: 0 if ((x >= q_25_reading) & (x <= q_75_reading))
                                   else 1 if (x > q_75_reading)
                                   else -1 if (x < q_25_reading)
                                   else -100)    

df['Label_All (19-21)'] = df['% LOSS All 2019 to 2021'].apply(lambda x: 0 if ((x >= q_25_all) & (x <= q_75_all))
                                   else 1 if (x > q_75_all)
                                   else -1 if (x < q_25_all)
                                   else -100)      

In [ ]:
mean_math = df['% LOSS Math 2021 to 2022'].mean()
std_math = df['% LOSS Math 2021 to 2022'].std()
q_25_math = df['% LOSS Math 2021 to 2022'].quantile(0.25)
q_75_math = df['% LOSS Math 2021 to 2022'].quantile(0.75)

mean_reading = df['% LOSS Reading 2021 to 2022'].mean()
std_reading = df['% LOSS Reading 2021 to 2022'].std()
q_25_reading = df['% LOSS Reading 2021 to 2022'].quantile(0.25)
q_75_reading = df['% LOSS Reading 2021 to 2022'].quantile(0.75)

mean_all = df['% LOSS All 2021 to 2022'].mean()
std_all = df['% LOSS All 2021 to 2022'].std()
q_25_all = df['% LOSS All 2021 to 2022'].quantile(0.25)
q_75_all = df['% LOSS All 2021 to 2022'].quantile(0.75)

sig = 1
sig_pos_math = mean_math + (sig * std_math)
sig_neg_math = mean_math - (sig * std_math)
sig_pos_reading = mean_reading + (sig * std_reading)
sig_neg_reading = mean_reading - (sig * std_reading)
sig_pos_all = mean_all + (sig * std_all)
sig_neg_all = mean_all - (sig * std_all)

df['Label_Math (21-22)'] = df['% LOSS Math 2021 to 2022'].apply(lambda x: 0 if ((x >= q_25_math) & (x <= q_75_math))
                                   else 1 if (x > q_75_math)
                                   else -1 if (x < q_25_math)
                                   else -100)   

df['Label_Reading (21-22)'] = df['% LOSS Reading 2021 to 2022'].apply(lambda x: 0 if ((x >= q_25_reading) & (x <= q_75_reading))
                                   else 1 if (x > q_75_reading)
                                   else -1 if (x < q_25_reading)
                                   else -100)    

df['Label_All (21-22)'] = df['% LOSS All 2021 to 2022'].apply(lambda x: 0 if ((x >= q_25_all) & (x <= q_75_all))
                                   else 1 if (x > q_75_all)
                                   else -1 if (x < q_25_all)
                                   else -100)      

In [ ]:
mean_math = df['% LOSS Math 2019 to 2022'].mean()
std_math = df['% LOSS Math 2019 to 2022'].std()
q_25_math = df['% LOSS Math 2019 to 2022'].quantile(0.25)
q_75_math = df['% LOSS Math 2019 to 2022'].quantile(0.75)

mean_reading = df['% LOSS Reading 2019 to 2022'].mean()
std_reading = df['% LOSS Reading 2019 to 2022'].std()
q_25_reading = df['% LOSS Reading 2019 to 2022'].quantile(0.25)
q_75_reading = df['% LOSS Reading 2019 to 2022'].quantile(0.75)

mean_all = df['% LOSS All 2019 to 2022'].mean()
std_all = df['% LOSS All 2019 to 2022'].std()
q_25_all = df['% LOSS All 2019 to 2022'].quantile(0.25)
q_75_all = df['% LOSS All 2019 to 2022'].quantile(0.75)

sig = 1
sig_pos_math = mean_math + (sig * std_math)
sig_neg_math = mean_math - (sig * std_math)
sig_pos_reading = mean_reading + (sig * std_reading)
sig_neg_reading = mean_reading - (sig * std_reading)
sig_pos_all = mean_all + (sig * std_all)
sig_neg_all = mean_all - (sig * std_all)

df['Label_Math (19-22)'] = df['% LOSS Math 2019 to 2022'].apply(lambda x: 0 if ((x >= q_25_math) & (x <= q_75_math))
                                   else 1 if (x > q_75_math)
                                   else -1 if (x < q_25_math)
                                   else -100)   

df['Label_Reading (19-22)'] = df['% LOSS Reading 2019 to 2022'].apply(lambda x: 0 if ((x >= q_25_reading) & (x <= q_75_reading))
                                   else 1 if (x > q_75_reading)
                                   else -1 if (x < q_25_reading)
                                   else -100)    

df['Label_All (19-22)'] = df['% LOSS All 2019 to 2022'].apply(lambda x: 0 if ((x >= q_25_all) & (x <= q_75_all))
                                   else 1 if (x > q_75_all)
                                   else -1 if (x < q_25_all)
                                   else -100)  

In [ ]:
df_math = df['Label_Math (19-21)'].value_counts().sort_index().to_frame()
df_math.reset_index(inplace=True)
df_math['label'] = df_math['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

df_reading = df['Label_Reading (19-21)'].value_counts().sort_index().to_frame()
df_reading.reset_index(inplace=True)
df_reading['label'] = df_reading['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

df_all = df['Label_All (19-21)'].value_counts().sort_index().to_frame()
df_all.reset_index(inplace=True)
df_all['label'] = df_all['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

fig, axes=plt.subplots(1,3, figsize=(20,10))
sns.set(font_scale=1.5)

colors=sns.color_palette('deep')[:df_all.shape[0]]
axes[0].pie(df_all['Label_All (19-21)'], labels=df_all['label'], colors=colors, autopct='%1.1f%%', radius=.8)
axes[0].set(title='All')

colors=sns.color_palette('pastel')[:df_math.shape[0]]
axes[1].pie(df_math['Label_Math (19-21)'], labels=df_math['label'], colors=colors, radius=.8,
            autopct=lambda p: '{:.0f}'.format(p*df_all['Label_All (19-21)'].sum()/100))
axes[1].set(title='Math')

colors=sns.color_palette('pastel')[:df_reading.shape[0]]
axes[2].pie(df_reading['Label_Reading (19-21)'], labels=df_reading['label'], colors=colors, radius=.8,
            autopct=lambda p: '{:.0f}'.format(p*df_all['Label_All (19-21)'].sum()/100))
axes[2].set(title='Reading')

plt.show()

In [ ]:
df_math = df['Label_Math (21-22)'].value_counts().sort_index().to_frame()
df_math.reset_index(inplace=True)
df_math['label'] = df_math['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

df_reading = df['Label_Reading (21-22)'].value_counts().sort_index().to_frame()
df_reading.reset_index(inplace=True)
df_reading['label'] = df_reading['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

df_all = df['Label_All (21-22)'].value_counts().sort_index().to_frame()
df_all.reset_index(inplace=True)
df_all['label'] = df_all['index'].apply(lambda x: '-1: Loss' if x == -1
                                         else '0: Expected' if x == 0
                                         else '1: Gain' if x == 1
                                         else '-100')

fig, axes=plt.subplots(1,3, figsize=(20,10))
sns.set(font_scale=1.5)

colors=sns.color_palette('deep')[:df_all.shape[0]]
axes[0].pie(df_all['Label_All (21-22)'], labels=df_all['label'], colors=colors, autopct='%1.1f%%', radius=.8)
axes[0].set(title='All')

colors=sns.color_palette('pastel')[:df_math.shape[0]]
axes[1].pie(df_math['Label_Math (21-22)'], labels=df_math['label'], colors=colors, radius=.8,
            autopct=lambda p: '{:.0f}'.format(p*df_all['Label_All (21-22)'].sum()/100))
axes[1].set(title='Math')

colors=sns.color_palette('pastel')[:df_reading.shape[0]]
axes[2].pie(df_reading['Label_Reading (21-22)'], labels=df_reading['label'], colors=colors, radius=.8,
            autopct=lambda p: '{:.0f}'.format(p*df_all['Label_All (21-22)'].sum()/100))
axes[2].set(title='Reading')

plt.show()

In [ ]:
df['Label_Reading (21-22)']

## Data Cleaning for ML Modeling ##  
Dropping instances with missing values or redundant columns that are normalized.

In [ ]:
cols_keep = ['District #', 'County #', 'Locale',
'Labor Force Diff',
'Employed Diff',
'Unemployed Level Diff',
'Unemployed Rate Diff',
'Median Age 10',
'Median Age Male 10',
'Median Age Female 10',
'# of Households 10',
'Avg Household Size 10',
'# of Families 10',
'Avg Family Size 10',
'# of Housing Units 10',
'County Population',
'CARES ESSER I 20',
'CRRSA ESSER II 21',
'ARP ESSER III 21',
'ESSER-SUPP 22',
'ESSER-SUPP 23',
'% County Infected 09/28/20',
'% County Deaths 09/28/20',
'% County Infected 10/30/20',
'% County Deaths 10/30/20',
'% County Infected 01/29/21',
'% County Deaths 01/29/21',
'% County Infected 07/28/2021',
'% County Deaths 07/28/2021',
'% County Infected 01/29/2022',
'% County Deaths 01/29/2022',
'% County Infected 07/29/2022',
'% County Deaths 07/29/2022',
'% County Infected 10/30/2022',
'% County Deaths 10/30/2022',
'% On Campus 09/28/20',
'% On Campus 10/30/20',
'% On Campus 01/29/21',
'% White Pop 10',
'% Black Pop 10',
'% Asian Pop 10',
'% Hispanic Pop 10',
'% Male Pop 10',
'% Female Pop 10',
'% Age 0-4 Pop 10',
'% Age 5-9 Pop 10',
'% Age 10-14 Pop 10',
'% Age 15-19 Pop 10',
'% Age 20-24 Pop 10',
'% Age 25-34 Pop 10',
'% Age 35-44 Pop 10',
'% Age 45-54 Pop 10',
'% Age 55-64 Pop 10',
'% Age 65-74 Pop 10',
'% Age 75-84 Pop 10',
'% Age 85-Up Pop 10',
'% HH 1 Male 10',
'% HH 1 Female 10',
'% HH Married-Child 10',
'% HH Married-noChild 10',
'% HH Male-Child 10',
'% HH Female-Child 10',
'% Housing Vacant 10',
'% Housing Owner Occup 10',
'% Housing Renter Occup 10',
#'Total Public Schools Diff',
#'Total Students Diff',
'% Total Operational Public Schools Diff 2019-2021',
'% School-wide Title I Diff 2019-2021',
'% Title I Eligible School Diff 2019-2021',
'Teachers:Students Diff 2019-2021',
'Staff:Students Diff 2019-2021',
'% Free or Reduced-price Lunch Diff 2019-2021',
'% Asian or Asian/Pacific Islander Students Diff 2019-2021',
'% Hispanic Students Diff 2019-2021',
'% Black or African American Students Diff 2019-2021',
'% White Students Diff 2019-2021',
'% Grades 1-8 Diff 2019-2021',
'% Grades 9-12 Diff 2019-2021',
'% Prek Diff 2019-2021',
'% K Diff 2019-2021',
'% ADA Diff 2019-2021',        
'% Total Operational Public Schools Diff 2019-2022',
'% School-wide Title I Diff 2019-2022',
'% Title I Eligible School Diff 2019-2022',
'Teachers:Students Diff 2019-2022',
'Staff:Students Diff 2019-2022',
'% Free or Reduced-price Lunch Diff 2019-2022',
'% Asian or Asian/Pacific Islander Students Diff 2019-2022',
'% Hispanic Students Diff 2019-2022',
'% Black or African American Students Diff 2019-2022',
'% White Students Diff 2019-2022',
'% Grades 1-8 Diff 2019-2022',
'% Grades 9-12 Diff 2019-2022',
'% Prek Diff 2019-2022',
'% K Diff 2019-2022',
'% ADA Diff 2019-2022',        
'% Students Tested Math - Grade 3 Diff 2019-2022',
'% Students Tested Reading - Grade 3 Diff 2019-2022',
'% Students Tested Math - Grade 4 Diff 2019-2022',
'% Students Tested Reading - Grade 4 Diff 2019-2022',
'% Students Tested Math - Grade 5 Diff 2019-2022',
'% Students Tested Reading - Grade 5 Diff 2019-2022',
'% Students Tested Math - Grade 6 Diff 2019-2022',
'% Students Tested Reading - Grade 6 Diff 2019-2022',
'% Students Tested Math - Grade 7 Diff 2019-2022',
'% Students Tested Reading - Grade 7 Diff 2019-2022',
'% Students Tested Math - Grade 8 Diff 2019-2022',
'% Students Tested Reading - Grade 8 Diff 2019-2022',

'Total Public Schools Diff 2021-2022',
'Total Students Diff 2021-2022',
'% Total Operational Public Schools Diff 2021-2022',
'% School-wide Title I Diff 2021-2022',
'% Title I Eligible School Diff 2021-2022',
'Teachers:Students Diff 2021-2022',
'Staff:Students Diff 2021-2022',
'% Free or Reduced-price Lunch Diff 2021-2022',
'% Asian or Asian/Pacific Islander Students Diff 2021-2022',
'% Hispanic Students Diff 2021-2022',
'% Black or African American Students Diff 2021-2022',
'% White Students Diff 2021-2022',
'% Grades 1-8 Diff 2021-2022',
'% Grades 9-12 Diff 2021-2022',
'% Prek Diff 2021-2022',
'% K Diff 2021-2022',
'% ADA Diff 2021-2022',
'% Students Tested Math - Grade 3 Diff 2019-2021',
'% Students Tested Reading - Grade 3 Diff 2019-2021',
'% Students Tested Math - Grade 4 Diff 2019-2021',
'% Students Tested Reading - Grade 4 Diff 2019-2021',
'% Students Tested Math - Grade 5 Diff 2019-2021',
'% Students Tested Reading - Grade 5 Diff 2019-2021',
'% Students Tested Math - Grade 6 Diff 2019-2021',
'% Students Tested Reading - Grade 6 Diff 2019-2021',
'% Students Tested Math - Grade 7 Diff 2019-2021',
'% Students Tested Reading - Grade 7 Diff 2019-2021',
'% Students Tested Math - Grade 8 Diff 2019-2021',
'% Students Tested Reading - Grade 8 Diff 2019-2021',
'% Students Tested Math - Grade 3 Diff 2021-2022',
'% Students Tested Reading - Grade 3 Diff 2021-2022',
'% Students Tested Math - Grade 4 Diff 2021-2022',
'% Students Tested Reading - Grade 4 Diff 2021-2022',
'% Students Tested Math - Grade 5 Diff 2021-2022',
'% Students Tested Reading - Grade 5 Diff 2021-2022',
'% Students Tested Math - Grade 6 Diff 2021-2022',
'% Students Tested Reading - Grade 6 Diff 2021-2022',
'% Students Tested Math - Grade 7 Diff 2021-2022',
'% Students Tested Reading - Grade 7 Diff 2021-2022',
'% Students Tested Math - Grade 8 Diff 2021-2022',
'% Students Tested Reading - Grade 8 Diff 2021-2022',    
'Label_Math (19-21)',
'Label_Reading (19-21)',
'Label_All (19-21)',
'Label_Math (21-22)',
'Label_Reading (21-22)',
'Label_All (21-22)',
'Label_Math (19-22)',
'Label_Reading (19-22)',
'Label_All (19-22)']

cols_new = ['District #', 'County #', 'Locale',
'Labor Force Diff',
'Employed Diff',
'Unemployed Level Diff',
'Unemployed Rate Diff',
'Median Age 10',
'Median Age Male 10',
'Median Age Female 10',
'# of Households 10',
'Avg Household Size 10',
'# of Families 10',
'Avg Family Size 10',
'# of Housing Units 10',
'County Population',
'CARES ESSER I 20',
'CRRSA ESSER II 21',
'ARP ESSER III 21',
'ESSER-SUPP 22',
'ESSER-SUPP 23',
'% County Infected 09/28/20',
'% County Deaths 09/28/20',
'% County Infected 10/30/20',
'% County Deaths 10/30/20',
'% County Infected 01/29/21',
'% County Deaths 01/29/21',
'% County Infected 07/28/2021',
'% County Deaths 07/28/2021',
'% County Infected 01/29/2022',
'% County Deaths 01/29/2022',
'% County Infected 07/29/2022',
'% County Deaths 07/29/2022',
'% County Infected 10/30/2022',
'% County Deaths 10/30/2022',
'% On Campus 09/28/20',
'% On Campus 10/30/20',
'% On Campus 01/29/21',
'% White Pop 10',
'% Black Pop 10',
'% Asian Pop 10',
'% Hispanic Pop 10',
'% Male Pop 10',
'% Female Pop 10',
'% Age 0-4 Pop 10',
'% Age 5-9 Pop 10',
'% Age 10-14 Pop 10',
'% Age 15-19 Pop 10',
'% Age 20-24 Pop 10',
'% Age 25-34 Pop 10',
'% Age 35-44 Pop 10',
'% Age 45-54 Pop 10',
'% Age 55-64 Pop 10',
'% Age 65-74 Pop 10',
'% Age 75-84 Pop 10',
'% Age 85-Up Pop 10',
'% HH 1 Male 10',
'% HH 1 Female 10',
'% HH Married-Child 10',
'% HH Married-noChild 10',
'% HH Male-Child 10',
'% HH Female-Child 10',
'% Housing Vacant 10',
'% Housing Owner Occup 10',
'% Housing Renter Occup 10',
#'Total Schools Diff',
#'Total Students Diff',
'% Total Operational Public Schools Diff 2019-2021',
'% School-wide Title I Diff 2019-2021',
'% Title I Eligible School Diff 2019-2021',
'Teachers:Students Diff 2019-2021',
'Staff:Students Diff 2019-2021',
'% Free or Reduced-price Lunch Diff 2019-2021',
'% Asian or Asian/Pacific Islander Students Diff 2019-2021',
'% Hispanic Students Diff 2019-2021',
'% Black or African American Students Diff 2019-2021',
'% White Students Diff 2019-2021',
'% Grades 1-8 Diff 2019-2021',
'% Grades 9-12 Diff 2019-2021',
'% Prek Diff 2019-2021',
'% K Diff 2019-2021',
'% ADA Diff 2019-2021',        
'Total Public Schools Diff 2021-2022',
'Total Students Diff 2021-2022',
'% Total Operational Public Schools Diff 2021-2022',
'% School-wide Title I Diff 2021-2022',
'% Title I Eligible School Diff 2021-2022',
'Teachers:Students Diff 2021-2022',
'Staff:Students Diff 2021-2022',
'% Free or Reduced-price Lunch Diff 2021-2022',
'% Asian or Asian/Pacific Islander Students Diff 2021-2022',
'% Hispanic Students Diff 2021-2022',
'% Black or African American Students Diff 2021-2022',
'% White Students Diff 2021-2022',
'% Grades 1-8 Diff 2021-2022',
'% Grades 9-12 Diff 2021-2022',
'% Prek Diff 2021-2022',
'% K Diff 2021-2022',
'% ADA Diff 2021-2022',
'% Students Tested Math - Grade 3 Diff 2019-2021',
'% Students Tested Reading - Grade 3 Diff 2019-2021',
'% Students Tested Math - Grade 4 Diff 2019-2021',
'% Students Tested Reading - Grade 4 Diff 2019-2021',
'% Students Tested Math - Grade 5 Diff 2019-2021',
'% Students Tested Reading - Grade 5 Diff 2019-2021',
'% Students Tested Math - Grade 6 Diff 2019-2021',
'% Students Tested Reading - Grade 6 Diff 2019-2021',
'% Students Tested Math - Grade 7 Diff 2019-2021',
'% Students Tested Reading - Grade 7 Diff 2019-2021',
'% Students Tested Math - Grade 8 Diff 2019-2021',
'% Students Tested Reading - Grade 8 Diff 2019-2021',
'% Students Tested Math - Grade 3 Diff 2021-2022',
'% Students Tested Reading - Grade 3 Diff 2021-2022',
'% Students Tested Math - Grade 4 Diff 2021-2022',
'% Students Tested Reading - Grade 4 Diff 2021-2022',
'% Students Tested Math - Grade 5 Diff 2021-2022',
'% Students Tested Reading - Grade 5 Diff 2021-2022',
'% Students Tested Math - Grade 6 Diff 2021-2022',
'% Students Tested Reading - Grade 6 Diff 2021-2022',
'% Students Tested Math - Grade 7 Diff 2021-2022',
'% Students Tested Reading - Grade 7 Diff 2021-2022',
'% Students Tested Math - Grade 8 Diff 2021-2022',
'% Students Tested Reading - Grade 8 Diff 2021-2022',    
'% Total Operational Public Schools Diff 2019-2022',
'% School-wide Title I Diff 2019-2022',
'% Title I Eligible School Diff 2019-2022',
'Teachers:Students Diff 2019-2022',
'Staff:Students Diff 2019-2022',
'% Free or Reduced-price Lunch Diff 2019-2022',
'% Asian or Asian/Pacific Islander Students Diff 2019-2022',
'% Hispanic Students Diff 2019-2022',
'% Black or African American Students Diff 2019-2022',
'% White Students Diff 2019-2022',
'% Grades 1-8 Diff 2019-2022',
'% Grades 9-12 Diff 2019-2022',
'% Prek Diff 2019-2022',
'% K Diff 2019-2022',
'% ADA Diff 2019-2022',   
'% Students Tested Math - Grade 3 Diff 2019-2022',
'% Students Tested Reading - Grade 3 Diff 2019-2022',
'% Students Tested Math - Grade 4 Diff 2019-2022',
'% Students Tested Reading - Grade 4 Diff 2019-2022',
'% Students Tested Math - Grade 5 Diff 2019-2022',
'% Students Tested Reading - Grade 5 Diff 2019-2022',
'% Students Tested Math - Grade 6 Diff 2019-2022',
'% Students Tested Reading - Grade 6 Diff 2019-2022',
'% Students Tested Math - Grade 7 Diff 2019-2022',
'% Students Tested Reading - Grade 7 Diff 2019-2022',
'% Students Tested Math - Grade 8 Diff 2019-2022',
'% Students Tested Reading - Grade 8 Diff 2019-2022',
'Label_Math (19-22)',
'Label_Reading (19-22)',
'Label_All (19-22)',
'Label_Math (19-21)',
'Label_Reading (19-21)',
'Label_All (19-21)',
'Label_Math (21-22)',
'Label_Reading (21-22)',
'Label_All (21-22)']

cols_v3=['County #',
'Total Operational Public Schools 2018-2019',
'Total Public Schools 2018-2019',
'Total Students 2018-2019',
'FTE Teachers 2018-2019',
'Total Staff 2018-2019',
'District #',
'Total Operational Public Schools 2020-2021',
'Total Public Schools 2020-2021',
'Total Students 2020-2021',
'FTE Teachers 2020-2021',
'Total Staff 2020-2021',
'Total Operational Public Schools 2021-2022',
'Total Public Schools 2021-2022',
'Total Students 2021-2022',
'FTE Teachers 2021-2022',
'Total Staff 2021-2022',
'Locale',
'Free or Reduced Lunch 2021-2022',
'Free or Reduced Lunch 2020-2021',
'Free or Reduced Lunch 2018-2019',
'Asian or Asian/Pacific Islander Students 2021-2022',
'Asian or Asian/Pacific Islander Students 2020-2021',
'Asian or Asian/Pacific Islander Students 2018-2019',
'Hispanic Students 2021-2022',
'Hispanic Students 2020-2021',
'Hispanic Students 2018-2019',
'Black or African American Students 2021-2022',
'Black or African American Students 2020-2021',
'Black or African American Students 2018-2019',
'White Students 2021-2022',        
'White Students 2020-2021',
'White Students 2018-2019',
'School-wide Title I 2021-2022',
'School-wide Title I 2020-2021',
'School-wide Title I 2018-2019',
'Title I Eligible School 2021-2022',
'Title I Eligible School 2020-2021',
'Title I Eligible School 2018-2019',
'Grades 1-8 2021-2022',
'Grades 1-8 2020-2021',
'Grades 1-8 2018-2019',
'Grades 9-12 2021-2022',
'Grades 9-12 2020-2021',
'Grades 9-12 2018-2019',
'Prek 2021-2022',         
'Prek 2020-2021',
'Prek 2018-2019',
'K 2021-2022',
'K 2020-2021',
'K 2018-2019',
'r_all_d_g3_2019',
'r_ethh_d_g3_2019',
'r_etha_d_g3_2019',
'r_ethb_d_g3_2019',
'r_ethw_d_g3_2019',
'r_ecoy_d_g3_2019',
'r_eco1_d_g3_2019',
'r_ti1y_d_g3_2019',
'r_spey_d_g3_2019',
'm_all_d_g3_2019',
'm_ethh_d_g3_2019',
'm_etha_d_g3_2019',
'm_ethb_d_g3_2019',
'm_ethw_d_g3_2019',
'm_ecoy_d_g3_2019',
'm_eco1_d_g3_2019',
'm_ti1y_d_g3_2019',
'm_spey_d_g3_2019',
'r_all_d_g4_2019',
'r_ethh_d_g4_2019',
'r_etha_d_g4_2019',
'r_ethb_d_g4_2019',
'r_ethw_d_g4_2019',
'r_ecoy_d_g4_2019',
'r_eco1_d_g4_2019',
'r_ti1y_d_g4_2019',
'r_spey_d_g4_2019',
'm_all_d_g4_2019',
'm_ethh_d_g4_2019',
'm_etha_d_g4_2019',
'm_ethb_d_g4_2019',
'm_ethw_d_g4_2019',
'm_ecoy_d_g4_2019',
'm_eco1_d_g4_2019',
'm_ti1y_d_g4_2019',
'm_spey_d_g4_2019',
'r_all_d_g5_2019',
'r_ethh_d_g5_2019',
'r_etha_d_g5_2019',
'r_ethb_d_g5_2019',
'r_ethw_d_g5_2019',
'r_ecoy_d_g5_2019',
'r_eco1_d_g5_2019',
'r_ti1y_d_g5_2019',
'r_spey_d_g5_2019',
'm_all_d_g5_2019',
'm_ethh_d_g5_2019',
'm_etha_d_g5_2019',
'm_ethb_d_g5_2019',
'm_ethw_d_g5_2019',
'm_ecoy_d_g5_2019',
'm_eco1_d_g5_2019',
'm_ti1y_d_g5_2019',
'm_spey_d_g5_2019',
'r_all_d_g6_2019',
'r_ethh_d_g6_2019',
'r_etha_d_g6_2019',
'r_ethb_d_g6_2019',
'r_ethw_d_g6_2019',
'r_ecoy_d_g6_2019',
'r_eco1_d_g6_2019',
'r_ti1y_d_g6_2019',
'r_spey_d_g6_2019',
'm_all_d_g6_2019',
'm_ethh_d_g6_2019',
'm_etha_d_g6_2019',
'm_ethb_d_g6_2019',
'm_ethw_d_g6_2019',
'm_ecoy_d_g6_2019',
'm_eco1_d_g6_2019',
'm_ti1y_d_g6_2019',
'm_spey_d_g6_2019',
'r_all_d_g7_2019',
'r_ethh_d_g7_2019',
'r_etha_d_g7_2019',
'r_ethb_d_g7_2019',
'r_ethw_d_g7_2019',
'r_ecoy_d_g7_2019',
'r_eco1_d_g7_2019',
'r_ti1y_d_g7_2019',
'r_spey_d_g7_2019',
'm_all_d_g7_2019',
'm_ethh_d_g7_2019',
'm_etha_d_g7_2019',
'm_ethb_d_g7_2019',
'm_ethw_d_g7_2019',
'm_ecoy_d_g7_2019',
'm_eco1_d_g7_2019',
'm_ti1y_d_g7_2019',
'm_spey_d_g7_2019',
'r_all_d_g8_2019',
'r_ethh_d_g8_2019',
'r_etha_d_g8_2019',
'r_ethb_d_g8_2019',
'r_ethw_d_g8_2019',
'r_ecoy_d_g8_2019',
'r_eco1_d_g8_2019',
'r_ti1y_d_g8_2019',
'r_spey_d_g8_2019',
'm_all_d_g8_2019',
'm_ethh_d_g8_2019',
'm_etha_d_g8_2019',
'm_ethb_d_g8_2019',
'm_ethw_d_g8_2019',
'm_ecoy_d_g8_2019',
'm_eco1_d_g8_2019',
'm_ti1y_d_g8_2019',
'm_spey_d_g8_2019',
'r_all_d_g3_2021',
'r_ethh_d_g3_2021',
'r_etha_d_g3_2021',
'r_ethb_d_g3_2021',
'r_ethw_d_g3_2021',
'r_ecoy_d_g3_2021',
'r_eco1_d_g3_2021',
'r_ti1y_d_g3_2021',
'r_spey_d_g3_2021',
'm_all_d_g3_2021',
'm_ethh_d_g3_2021',
'm_etha_d_g3_2021',
'm_ethb_d_g3_2021',
'm_ethw_d_g3_2021',
'm_ecoy_d_g3_2021',
'm_eco1_d_g3_2021',
'm_ti1y_d_g3_2021',
'm_spey_d_g3_2021',
'r_all_d_g4_2021',
'r_ethh_d_g4_2021',
'r_etha_d_g4_2021',
'r_ethb_d_g4_2021',
'r_ethw_d_g4_2021',
'r_ecoy_d_g4_2021',
'r_eco1_d_g4_2021',
'r_ti1y_d_g4_2021',
'r_spey_d_g4_2021',
'm_all_d_g4_2021',
'm_ethh_d_g4_2021',
'm_etha_d_g4_2021',
'm_ethb_d_g4_2021',
'm_ethw_d_g4_2021',
'm_ecoy_d_g4_2021',
'm_eco1_d_g4_2021',
'm_ti1y_d_g4_2021',
'm_spey_d_g4_2021',
'r_all_d_g5_2021',
'r_ethh_d_g5_2021',
'r_etha_d_g5_2021',
'r_ethb_d_g5_2021',
'r_ethw_d_g5_2021',
'r_ecoy_d_g5_2021',
'r_eco1_d_g5_2021',
'r_ti1y_d_g5_2021',
'r_spey_d_g5_2021',
'm_all_d_g5_2021',
'm_ethh_d_g5_2021',
'm_etha_d_g5_2021',
'm_ethb_d_g5_2021',
'm_ethw_d_g5_2021',
'm_ecoy_d_g5_2021',
'm_eco1_d_g5_2021',
'm_ti1y_d_g5_2021',
'm_spey_d_g5_2021',
'r_all_d_g6_2021',
'r_ethh_d_g6_2021',
'r_etha_d_g6_2021',
'r_ethb_d_g6_2021',
'r_ethw_d_g6_2021',
'r_ecoy_d_g6_2021',
'r_eco1_d_g6_2021',
'r_ti1y_d_g6_2021',
'r_spey_d_g6_2021',
'm_all_d_g6_2021',
'm_ethh_d_g6_2021',
'm_etha_d_g6_2021',
'm_ethb_d_g6_2021',
'm_ethw_d_g6_2021',
'm_ecoy_d_g6_2021',
'm_eco1_d_g6_2021',
'm_ti1y_d_g6_2021',
'm_spey_d_g6_2021',
'r_all_d_g7_2021',
'r_ethh_d_g7_2021',
'r_etha_d_g7_2021',
'r_ethb_d_g7_2021',
'r_ethw_d_g7_2021',
'r_ecoy_d_g7_2021',
'r_eco1_d_g7_2021',
'r_ti1y_d_g7_2021',
'r_spey_d_g7_2021',
'm_all_d_g7_2021',
'm_ethh_d_g7_2021',
'm_etha_d_g7_2021',
'm_ethb_d_g7_2021',
'm_ethw_d_g7_2021',
'm_ecoy_d_g7_2021',
'm_eco1_d_g7_2021',
'm_ti1y_d_g7_2021',
'm_spey_d_g7_2021',
'r_all_d_g8_2021',
'r_ethh_d_g8_2021',
'r_etha_d_g8_2021',
'r_ethb_d_g8_2021',
'r_ethw_d_g8_2021',
'r_ecoy_d_g8_2021',
'r_eco1_d_g8_2021',
'r_ti1y_d_g8_2021',
'r_spey_d_g8_2021',
'm_all_d_g8_2021',
'm_ethh_d_g8_2021',
'm_etha_d_g8_2021',
'm_ethb_d_g8_2021',
'm_ethw_d_g8_2021',
'm_ecoy_d_g8_2021',
'm_eco1_d_g8_2021',
'm_ti1y_d_g8_2021',
'm_spey_d_g8_2021',
'r_all_d_g3_2022',
'r_ethh_d_g3_2022',
'r_etha_d_g3_2022',
'r_ethb_d_g3_2022',
'r_ethw_d_g3_2022',
'r_ecoy_d_g3_2022',
'r_eco1_d_g3_2022',
'r_ti1y_d_g3_2022',
'r_spey_d_g3_2022',
'm_all_d_g3_2022',
'm_ethh_d_g3_2022',
'm_etha_d_g3_2022',
'm_ethb_d_g3_2022',
'm_ethw_d_g3_2022',
'm_ecoy_d_g3_2022',
'm_eco1_d_g3_2022',
'm_ti1y_d_g3_2022',
'm_spey_d_g3_2022',
'r_all_d_g4_2022',
'r_ethh_d_g4_2022',
'r_etha_d_g4_2022',
'r_ethb_d_g4_2022',
'r_ethw_d_g4_2022',
'r_ecoy_d_g4_2022',
'r_eco1_d_g4_2022',
'r_ti1y_d_g4_2022',
'r_spey_d_g4_2022',
'm_all_d_g4_2022',
'm_ethh_d_g4_2022',
'm_etha_d_g4_2022',
'm_ethb_d_g4_2022',
'm_ethw_d_g4_2022',
'm_ecoy_d_g4_2022',
'm_eco1_d_g4_2022',
'm_ti1y_d_g4_2022',
'm_spey_d_g4_2022',
'r_all_d_g5_2022',
'r_ethh_d_g5_2022',
'r_etha_d_g5_2022',
'r_ethb_d_g5_2022',
'r_ethw_d_g5_2022',
'r_ecoy_d_g5_2022',
'r_eco1_d_g5_2022',
'r_ti1y_d_g5_2022',
'r_spey_d_g5_2022',
'm_all_d_g5_2022',
'm_ethh_d_g5_2022',
'm_etha_d_g5_2022',
'm_ethb_d_g5_2022',
'm_ethw_d_g5_2022',
'm_ecoy_d_g5_2022',
'm_eco1_d_g5_2022',
'm_ti1y_d_g5_2022',
'm_spey_d_g5_2022',
'r_all_d_g6_2022',
'r_ethh_d_g6_2022',
'r_etha_d_g6_2022',
'r_ethb_d_g6_2022',
'r_ethw_d_g6_2022',
'r_ecoy_d_g6_2022',
'r_eco1_d_g6_2022',
'r_ti1y_d_g6_2022',
'r_spey_d_g6_2022',
'm_all_d_g6_2022',
'm_ethh_d_g6_2022',
'm_etha_d_g6_2022',
'm_ethb_d_g6_2022',
'm_ethw_d_g6_2022',
'm_ecoy_d_g6_2022',
'm_eco1_d_g6_2022',
'm_ti1y_d_g6_2022',
'm_spey_d_g6_2022',
'r_all_d_g7_2022',
'r_ethh_d_g7_2022',
'r_etha_d_g7_2022',
'r_ethb_d_g7_2022',
'r_ethw_d_g7_2022',
'r_ecoy_d_g7_2022',
'r_eco1_d_g7_2022',
'r_ti1y_d_g7_2022',
'r_spey_d_g7_2022',
'm_all_d_g7_2022',
'm_ethh_d_g7_2022',
'm_etha_d_g7_2022',
'm_ethb_d_g7_2022',
'm_ethw_d_g7_2022',
'm_ecoy_d_g7_2022',
'm_eco1_d_g7_2022',
'm_ti1y_d_g7_2022',
'm_spey_d_g7_2022',
'r_all_d_g8_2022',
'r_ethh_d_g8_2022',
'r_etha_d_g8_2022',
'r_ethb_d_g8_2022',
'r_ethw_d_g8_2022',
'r_ecoy_d_g8_2022',
'r_eco1_d_g8_2022',
'r_ti1y_d_g8_2022',
'r_spey_d_g8_2022',
'm_all_d_g8_2022',
'm_ethh_d_g8_2022',
'm_etha_d_g8_2022',
'm_ethb_d_g8_2022',
'm_ethw_d_g8_2022',
'm_ecoy_d_g8_2022',
'm_eco1_d_g8_2022',
'm_ti1y_d_g8_2022',
'm_spey_d_g8_2022',
'Labor Force 19',
'Employed 19',
'Unemployed Level 19',
'Unemployed Rate 19',
'Labor Force 21',
'Employed 21',
'Unemployed Level 21',
'Unemployed Rate 21',
'White Pop 10',
'Black Pop 10',
'Asian Pop 10',
'Hispanic Pop 10',
'Male Pop 10',
'Female Pop 10',
'Age 0-4 Pop 10',
'Age 5-9 Pop 10',
'Age 10-14 Pop 10',
'Age 15-19 Pop 10',
'Age 20-24 Pop 10',
'Age 25-34 Pop 10',
'Age 35-44 Pop 10',
'Age 45-54 Pop 10',
'Age 55-64 Pop 10',
'Age 65-74 Pop 10',
'Age 75-84 Pop 10',
'Age 85-Up Pop 10',
'Median Age 10',
'Median Age Male 10',
'Median Age Female 10',
'# of Households 10',
'Avg Household Size 10',
'HH 1 Male 10',
'HH 1 Female 10',
'HH Married-Child 10',
'HH Married-noChild 10',
'HH Male-Child 10',
'HH Female-Child 10',
'# of Families 10',
'Avg Family Size 10',
'# of Housing Units 10',
'Housing Vacant 10',
'Housing Owner Occup 10',
'Housing Renter Occup 10',
'District Enrollment 01/29/21',
'Approx. District On-Campus Enrollment 01/29/21',
'District Enrollment 10/30/20',
'Approx. District On-Campus Enrollment 10/30/20',
'District Enrollment 09/28/20',
'Approx. District On-Campus Enrollment 09/28/20',
'Confirmed Deaths 09/28/20',
'Confirmed Deaths 10/30/20',
'Confirmed Deaths 01/29/21',
'Confirmed Deaths 07/28/2021',
'Confirmed Deaths 01/29/2022',
'Confirmed Deaths 07/29/2022',
'Confirmed Deaths 10/30/2022',
'Confirmed Cases 09/28/20',
'Confirmed Cases 10/30/20',
'Confirmed Cases 01/29/21',
'Confirmed Cases 07/28/2021',
'Confirmed Cases 01/29/2022',
'Confirmed Cases 07/29/2022',
'Confirmed Cases 10/30/2022',
'County Population',
'2018-2019 ADA',
'2020-2021 ADA',
'2021-2022 ADA',
'CARES ESSER I 20',
'CRRSA ESSER II 21',
'ARP ESSER III 21',
'ESSER-SUPP 22',
'ESSER-SUPP 23',
'Average Score Math Hispanic Grade 3 2018-2019',
'Average Score Reading Hispanic Grade 3 2018-2019',
'Average Score Math Asian Grade 3 2018-2019',
'Average Score Reading Asian Grade 3 2018-2019',
'Average Score Math Black Grade 3 2018-2019',
'Average Score Reading Black Grade 3 2018-2019',
'Average Score Math White Grade 3 2018-2019',
'Average Score Reading White Grade 3 2018-2019',
'Average Score Math Poverty Grade 3 2018-2019',
'Average Score Reading Poverty Grade 3 2018-2019',
'Average Score Math Free Lunch Grade 3 2018-2019',
'Average Score Reading Free Lunch Grade 3 2018-2019',
'Average Score Math T1 Grade 3 2018-2019',
'Average Score Reading T1 Grade 3 2018-2019',
'Average Score Math Special Ed Grade 3 2018-2019',
'Average Score Reading Special Ed Grade 3 2018-2019',
'Average Score Math Hispanic Grade 4 2018-2019',
'Average Score Reading Hispanic Grade 4 2018-2019',
'Average Score Math Asian Grade 4 2018-2019',
'Average Score Reading Asian Grade 4 2018-2019',
'Average Score Math Black Grade 4 2018-2019',
'Average Score Reading Black Grade 4 2018-2019',
'Average Score Math White Grade 4 2018-2019',
'Average Score Reading White Grade 4 2018-2019',
'Average Score Math Poverty Grade 4 2018-2019',
'Average Score Reading Poverty Grade 4 2018-2019',
'Average Score Math Free Lunch Grade 4 2018-2019',
'Average Score Reading Free Lunch Grade 4 2018-2019',
'Average Score Math T1 Grade 4 2018-2019',
'Average Score Reading T1 Grade 4 2018-2019',
'Average Score Math Special Ed Grade 4 2018-2019',
'Average Score Reading Special Ed Grade 4 2018-2019',
'Average Score Math Hispanic Grade 5 2018-2019',
'Average Score Reading Hispanic Grade 5 2018-2019',
'Average Score Math Asian Grade 5 2018-2019',
'Average Score Reading Asian Grade 5 2018-2019',
'Average Score Math Black Grade 5 2018-2019',
'Average Score Reading Black Grade 5 2018-2019',
'Average Score Math White Grade 5 2018-2019',
'Average Score Reading White Grade 5 2018-2019',
'Average Score Math Poverty Grade 5 2018-2019',
'Average Score Reading Poverty Grade 5 2018-2019',
'Average Score Math Free Lunch Grade 5 2018-2019',
'Average Score Reading Free Lunch Grade 5 2018-2019',
'Average Score Math T1 Grade 5 2018-2019',
'Average Score Reading T1 Grade 5 2018-2019',
'Average Score Math Special Ed Grade 5 2018-2019',
'Average Score Reading Special Ed Grade 5 2018-2019',
'Average Score Math Hispanic Grade 6 2018-2019',
'Average Score Reading Hispanic Grade 6 2018-2019',
'Average Score Math Asian Grade 6 2018-2019',
'Average Score Reading Asian Grade 6 2018-2019',
'Average Score Math Black Grade 6 2018-2019',
'Average Score Reading Black Grade 6 2018-2019',
'Average Score Math White Grade 6 2018-2019',
'Average Score Reading White Grade 6 2018-2019',
'Average Score Math Poverty Grade 6 2018-2019',
'Average Score Reading Poverty Grade 6 2018-2019',
'Average Score Math Free Lunch Grade 6 2018-2019',
'Average Score Reading Free Lunch Grade 6 2018-2019',
'Average Score Math T1 Grade 6 2018-2019',
'Average Score Reading T1 Grade 6 2018-2019',
'Average Score Math Special Ed Grade 6 2018-2019',
'Average Score Reading Special Ed Grade 6 2018-2019',
'Average Score Math Hispanic Grade 7 2018-2019',
'Average Score Reading Hispanic Grade 7 2018-2019',
'Average Score Math Asian Grade 7 2018-2019',
'Average Score Reading Asian Grade 7 2018-2019',
'Average Score Math Black Grade 7 2018-2019',
'Average Score Reading Black Grade 7 2018-2019',
'Average Score Math White Grade 7 2018-2019',
'Average Score Reading White Grade 7 2018-2019',
'Average Score Math Poverty Grade 7 2018-2019',
'Average Score Reading Poverty Grade 7 2018-2019',
'Average Score Math Free Lunch Grade 7 2018-2019',
'Average Score Reading Free Lunch Grade 7 2018-2019',
'Average Score Math T1 Grade 7 2018-2019',
'Average Score Reading T1 Grade 7 2018-2019',
'Average Score Math Special Ed Grade 7 2018-2019',
'Average Score Reading Special Ed Grade 7 2018-2019',
'Average Score Math Hispanic Grade 8 2018-2019',
'Average Score Reading Hispanic Grade 8 2018-2019',
'Average Score Math Asian Grade 8 2018-2019',
'Average Score Reading Asian Grade 8 2018-2019',
'Average Score Math Black Grade 8 2018-2019',
'Average Score Reading Black Grade 8 2018-2019',
'Average Score Math White Grade 8 2018-2019',
'Average Score Reading White Grade 8 2018-2019',
'Average Score Math Poverty Grade 8 2018-2019',
'Average Score Reading Poverty Grade 8 2018-2019',
'Average Score Math Free Lunch Grade 8 2018-2019',
'Average Score Reading Free Lunch Grade 8 2018-2019',
'Average Score Math T1 Grade 8 2018-2019',
'Average Score Reading T1 Grade 8 2018-2019',
'Average Score Math Special Ed Grade 8 2018-2019',
'Average Score Reading Special Ed Grade 8 2018-2019',
'Average Score Math Hispanic Grade 3 2020-2021',
'Average Score Reading Hispanic Grade 3 2020-2021',
'Average Score Math Asian Grade 3 2020-2021',
'Average Score Reading Asian Grade 3 2020-2021',
'Average Score Math Black Grade 3 2020-2021',
'Average Score Reading Black Grade 3 2020-2021',
'Average Score Math White Grade 3 2020-2021',
'Average Score Reading White Grade 3 2020-2021',
'Average Score Math Poverty Grade 3 2020-2021',
'Average Score Reading Poverty Grade 3 2020-2021',
'Average Score Math Free Lunch Grade 3 2020-2021',
'Average Score Reading Free Lunch Grade 3 2020-2021',
'Average Score Math T1 Grade 3 2020-2021',
'Average Score Reading T1 Grade 3 2020-2021',
'Average Score Math Special Ed Grade 3 2020-2021',
'Average Score Reading Special Ed Grade 3 2020-2021',
'Average Score Math Hispanic Grade 4 2020-2021',
'Average Score Reading Hispanic Grade 4 2020-2021',
'Average Score Math Asian Grade 4 2020-2021',
'Average Score Reading Asian Grade 4 2020-2021',
'Average Score Math Black Grade 4 2020-2021',
'Average Score Reading Black Grade 4 2020-2021',
'Average Score Math White Grade 4 2020-2021',
'Average Score Reading White Grade 4 2020-2021',
'Average Score Math Poverty Grade 4 2020-2021',
'Average Score Reading Poverty Grade 4 2020-2021',
'Average Score Math Free Lunch Grade 4 2020-2021',
'Average Score Reading Free Lunch Grade 4 2020-2021',
'Average Score Math T1 Grade 4 2020-2021',
'Average Score Reading T1 Grade 4 2020-2021',
'Average Score Math Special Ed Grade 4 2020-2021',
'Average Score Reading Special Ed Grade 4 2020-2021',
'Average Score Math Hispanic Grade 5 2020-2021',
'Average Score Reading Hispanic Grade 5 2020-2021',
'Average Score Math Asian Grade 5 2020-2021',
'Average Score Reading Asian Grade 5 2020-2021',
'Average Score Math Black Grade 5 2020-2021',
'Average Score Reading Black Grade 5 2020-2021',
'Average Score Math White Grade 5 2020-2021',
'Average Score Reading White Grade 5 2020-2021',
'Average Score Math Poverty Grade 5 2020-2021',
'Average Score Reading Poverty Grade 5 2020-2021',
'Average Score Math Free Lunch Grade 5 2020-2021',
'Average Score Reading Free Lunch Grade 5 2020-2021',
'Average Score Math T1 Grade 5 2020-2021',
'Average Score Reading T1 Grade 5 2020-2021',
'Average Score Math Special Ed Grade 5 2020-2021',
'Average Score Reading Special Ed Grade 5 2020-2021',
'Average Score Math Hispanic Grade 6 2020-2021',
'Average Score Reading Hispanic Grade 6 2020-2021',
'Average Score Math Asian Grade 6 2020-2021',
'Average Score Reading Asian Grade 6 2020-2021',
'Average Score Math Black Grade 6 2020-2021',
'Average Score Reading Black Grade 6 2020-2021',
'Average Score Math White Grade 6 2020-2021',
'Average Score Reading White Grade 6 2020-2021',
'Average Score Math Poverty Grade 6 2020-2021',
'Average Score Reading Poverty Grade 6 2020-2021',
'Average Score Math Free Lunch Grade 6 2020-2021',
'Average Score Reading Free Lunch Grade 6 2020-2021',
'Average Score Math T1 Grade 6 2020-2021',
'Average Score Reading T1 Grade 6 2020-2021',
'Average Score Math Special Ed Grade 6 2020-2021',
'Average Score Reading Special Ed Grade 6 2020-2021',
'Average Score Math Hispanic Grade 7 2020-2021',
'Average Score Reading Hispanic Grade 7 2020-2021',
'Average Score Math Asian Grade 7 2020-2021',
'Average Score Reading Asian Grade 7 2020-2021',
'Average Score Math Black Grade 7 2020-2021',
'Average Score Reading Black Grade 7 2020-2021',
'Average Score Math White Grade 7 2020-2021',
'Average Score Reading White Grade 7 2020-2021',
'Average Score Math Poverty Grade 7 2020-2021',
'Average Score Reading Poverty Grade 7 2020-2021',
'Average Score Math Free Lunch Grade 7 2020-2021',
'Average Score Reading Free Lunch Grade 7 2020-2021',
'Average Score Math T1 Grade 7 2020-2021',
'Average Score Reading T1 Grade 7 2020-2021',
'Average Score Math Special Ed Grade 7 2020-2021',
'Average Score Reading Special Ed Grade 7 2020-2021',
'Average Score Math Hispanic Grade 8 2020-2021',
'Average Score Reading Hispanic Grade 8 2020-2021',
'Average Score Math Asian Grade 8 2020-2021',
'Average Score Reading Asian Grade 8 2020-2021',
'Average Score Math Black Grade 8 2020-2021',
'Average Score Reading Black Grade 8 2020-2021',
'Average Score Math White Grade 8 2020-2021',
'Average Score Reading White Grade 8 2020-2021',
'Average Score Math Poverty Grade 8 2020-2021',
'Average Score Reading Poverty Grade 8 2020-2021',
'Average Score Math Free Lunch Grade 8 2020-2021',
'Average Score Reading Free Lunch Grade 8 2020-2021',
'Average Score Math T1 Grade 8 2020-2021',
'Average Score Reading T1 Grade 8 2020-2021',
'Average Score Math Special Ed Grade 8 2020-2021',
'Average Score Reading Special Ed Grade 8 2020-2021',
'Average Score Math Hispanic Grade 3 2021-2022',
'Average Score Reading Hispanic Grade 3 2021-2022',
'Average Score Math Asian Grade 3 2021-2022',
'Average Score Reading Asian Grade 3 2021-2022',
'Average Score Math Black Grade 3 2021-2022',
'Average Score Reading Black Grade 3 2021-2022',
'Average Score Math White Grade 3 2021-2022',
'Average Score Reading White Grade 3 2021-2022',
'Average Score Math Poverty Grade 3 2021-2022',
'Average Score Reading Poverty Grade 3 2021-2022',
'Average Score Math Free Lunch Grade 3 2021-2022',
'Average Score Reading Free Lunch Grade 3 2021-2022',
'Average Score Math T1 Grade 3 2021-2022',
'Average Score Reading T1 Grade 3 2021-2022',
'Average Score Math Special Ed Grade 3 2021-2022',
'Average Score Reading Special Ed Grade 3 2021-2022',
'Average Score Math Hispanic Grade 4 2021-2022',
'Average Score Reading Hispanic Grade 4 2021-2022',
'Average Score Math Asian Grade 4 2021-2022',
'Average Score Reading Asian Grade 4 2021-2022',
'Average Score Math Black Grade 4 2021-2022',
'Average Score Reading Black Grade 4 2021-2022',
'Average Score Math White Grade 4 2021-2022',
'Average Score Reading White Grade 4 2021-2022',
'Average Score Math Poverty Grade 4 2021-2022',
'Average Score Reading Poverty Grade 4 2021-2022',
'Average Score Math Free Lunch Grade 4 2021-2022',
'Average Score Reading Free Lunch Grade 4 2021-2022',
'Average Score Math T1 Grade 4 2021-2022',
'Average Score Reading T1 Grade 4 2021-2022',
'Average Score Math Special Ed Grade 4 2021-2022',
'Average Score Reading Special Ed Grade 4 2021-2022',
'Average Score Math Hispanic Grade 5 2021-2022',
'Average Score Reading Hispanic Grade 5 2021-2022',
'Average Score Math Asian Grade 5 2021-2022',
'Average Score Reading Asian Grade 5 2021-2022',
'Average Score Math Black Grade 5 2021-2022',
'Average Score Reading Black Grade 5 2021-2022',
'Average Score Math White Grade 5 2021-2022',
'Average Score Reading White Grade 5 2021-2022',
'Average Score Math Poverty Grade 5 2021-2022',
'Average Score Reading Poverty Grade 5 2021-2022',
'Average Score Math Free Lunch Grade 5 2021-2022',
'Average Score Reading Free Lunch Grade 5 2021-2022',
'Average Score Math T1 Grade 5 2021-2022',
'Average Score Reading T1 Grade 5 2021-2022',
'Average Score Math Special Ed Grade 5 2021-2022',
'Average Score Reading Special Ed Grade 5 2021-2022',
'Average Score Math Hispanic Grade 6 2021-2022',
'Average Score Reading Hispanic Grade 6 2021-2022',
'Average Score Math Asian Grade 6 2021-2022',
'Average Score Reading Asian Grade 6 2021-2022',
'Average Score Math Black Grade 6 2021-2022',
'Average Score Reading Black Grade 6 2021-2022',
'Average Score Math White Grade 6 2021-2022',
'Average Score Reading White Grade 6 2021-2022',
'Average Score Math Poverty Grade 6 2021-2022',
'Average Score Reading Poverty Grade 6 2021-2022',
'Average Score Math Free Lunch Grade 6 2021-2022',
'Average Score Reading Free Lunch Grade 6 2021-2022',
'Average Score Math T1 Grade 6 2021-2022',
'Average Score Reading T1 Grade 6 2021-2022',
'Average Score Math Special Ed Grade 6 2021-2022',
'Average Score Reading Special Ed Grade 6 2021-2022',
'Average Score Math Hispanic Grade 7 2021-2022',
'Average Score Reading Hispanic Grade 7 2021-2022',
'Average Score Math Asian Grade 7 2021-2022',
'Average Score Reading Asian Grade 7 2021-2022',
'Average Score Math Black Grade 7 2021-2022',
'Average Score Reading Black Grade 7 2021-2022',
'Average Score Math White Grade 7 2021-2022',
'Average Score Reading White Grade 7 2021-2022',
'Average Score Math Poverty Grade 7 2021-2022',
'Average Score Reading Poverty Grade 7 2021-2022',
'Average Score Math Free Lunch Grade 7 2021-2022',
'Average Score Reading Free Lunch Grade 7 2021-2022',
'Average Score Math T1 Grade 7 2021-2022',
'Average Score Reading T1 Grade 7 2021-2022',
'Average Score Math Special Ed Grade 7 2021-2022',
'Average Score Reading Special Ed Grade 7 2021-2022',
'Average Score Math Hispanic Grade 8 2021-2022',
'Average Score Reading Hispanic Grade 8 2021-2022',
'Average Score Math Asian Grade 8 2021-2022',
'Average Score Reading Asian Grade 8 2021-2022',
'Average Score Math Black Grade 8 2021-2022',
'Average Score Reading Black Grade 8 2021-2022',
'Average Score Math White Grade 8 2021-2022',
'Average Score Reading White Grade 8 2021-2022',
'Average Score Math Poverty Grade 8 2021-2022',
'Average Score Reading Poverty Grade 8 2021-2022',
'Average Score Math Free Lunch Grade 8 2021-2022',
'Average Score Reading Free Lunch Grade 8 2021-2022',
'Average Score Math T1 Grade 8 2021-2022',
'Average Score Reading T1 Grade 8 2021-2022',
'Average Score Math Special Ed Grade 8 2021-2022',
'Average Score Reading Special Ed Grade 8 2021-2022',
'Label_Math (19-21)',
'Label_Reading (19-21)',
'Label_All (19-21)',
'Label_Math (21-22)',
'Label_Reading (21-22)',
'Label_All (21-22)',
'Label_Math (19-22)',
'Label_Reading (19-22)',
'Label_All (19-22)']

print(len(cols_keep), len(cols_new))
print(len(cols_v3))

In [ ]:
cols_new19 = ['District #', 'County #', 'Locale',
'Labor Force Diff',
'Employed Diff',
'Unemployed Level Diff',
'Unemployed Rate Diff',
'Median Age 10',
'Median Age Male 10',
'Median Age Female 10',
'# of Households 10',
'Avg Household Size 10',
'# of Families 10',
'Avg Family Size 10',
'# of Housing Units 10',
'County Population',
'CARES ESSER I 20',
'CRRSA ESSER II 21',
'ARP ESSER III 21',
'ESSER-SUPP 22',
'ESSER-SUPP 23',
'% County Infected 09/28/20',
'% County Deaths 09/28/20',
'% County Infected 10/30/20',
'% County Deaths 10/30/20',
'% County Infected 01/29/21',
'% County Deaths 01/29/21',
'% On Campus 09/28/20',
'% On Campus 10/30/20',
'% On Campus 01/29/21',
'% White Pop 10',
'% Black Pop 10',
'% Asian Pop 10',
'% Hispanic Pop 10',
'% Male Pop 10',
'% Female Pop 10',
'% Age 0-4 Pop 10',
'% Age 5-9 Pop 10',
'% Age 10-14 Pop 10',
'% Age 15-19 Pop 10',
'% Age 20-24 Pop 10',
'% Age 25-34 Pop 10',
'% Age 35-44 Pop 10',
'% Age 45-54 Pop 10',
'% Age 55-64 Pop 10',
'% Age 65-74 Pop 10',
'% Age 75-84 Pop 10',
'% Age 85-Up Pop 10',
'% HH 1 Male 10',
'% HH 1 Female 10',
'% HH Married-Child 10',
'% HH Married-noChild 10',
'% HH Male-Child 10',
'% HH Female-Child 10',
'% Housing Vacant 10',
'% Housing Owner Occup 10',
'% Housing Renter Occup 10',
#'Total Schools Diff',
#'Total Students Diff',
'% Total Operational Public Schools Diff 2019-2021',
'% School-wide Title I Diff 2019-2021',
'% Title I Eligible School Diff 2019-2021',
'Teachers:Students Diff 2019-2021',
'Staff:Students Diff 2019-2021',
'% Free or Reduced-price Lunch Diff 2019-2021',
'% Asian or Asian/Pacific Islander Students Diff 2019-2021',
'% Hispanic Students Diff 2019-2021',
'% Black or African American Students Diff 2019-2021',
'% White Students Diff 2019-2021',
'% Grades 1-8 Diff 2019-2021',
'% Grades 9-12 Diff 2019-2021',
'% Prek Diff 2019-2021',
'% K Diff 2019-2021',
'% ADA Diff 2019-2021',        
'% Students Tested Math - Grade 3 Diff 2019-2021',
'% Students Tested Reading - Grade 3 Diff 2019-2021',
'% Students Tested Math - Grade 4 Diff 2019-2021',
'% Students Tested Reading - Grade 4 Diff 2019-2021',
'% Students Tested Math - Grade 5 Diff 2019-2021',
'% Students Tested Reading - Grade 5 Diff 2019-2021',
'% Students Tested Math - Grade 6 Diff 2019-2021',
'% Students Tested Reading - Grade 6 Diff 2019-2021',
'% Students Tested Math - Grade 7 Diff 2019-2021',
'% Students Tested Reading - Grade 7 Diff 2019-2021',
'% Students Tested Math - Grade 8 Diff 2019-2021',
'% Students Tested Reading - Grade 8 Diff 2019-2021',
'Label_Math (19-21)',
'Label_Reading (19-21)',
'Label_All (19-21)',
]

In [ ]:
print('Shape:', df.shape)
temp=[c + ': ' + str(df[c].isna().sum()) for c in df[cols_keep] if df[c].isna().any()]
print("na rows:", len(temp))
temp

### Handing columns with erroneous values ###   
- Fixing any instance that `% 2018-2019 ADA`, `% 2020-2021 ADA`, `% 2021-2022 ADA` are larger than 1 meaning the Daily Average Attendance `2018-2019 ADA`, `2020-2021 ADA`, `2021-2022 ADA` from ADA are larger than the Total Students `Total Students 2018-2019`, `Total Students 2020-2021`, (eventually `Total Students 2021-2022`) from NCES.

In [ ]:
print("{} rows are updated to 1 from '% ADA 2018-2019'".format(df[df['% ADA 2018-2019'] > 1].shape[0]))
df['% ADA 2018-2019'] = df['% ADA 2018-2019'].apply(lambda x: 1 if x>1 else x)
print("{} rows are updated to 1 from '% ADA 2020-2021'".format(df[df['% ADA 2020-2021'] > 1].shape[0]))
df['% ADA 2020-2021'] = df['% ADA 2020-2021'].apply(lambda x: 1 if x>1 else x)
print("{} rows are updated to 1 from '% ADA 2021-2022'".format(df[df['% ADA 2021-2022'] > 1].shape[0]))
df['% ADA 2021-2022'] = df['% ADA 2021-2022'].apply(lambda x: 1 if x>1 else x)

### Exporting the integrated dataframe into 3 csv files:

- DATA_Texas_District_v1.csv: raw integrated data with normalization, delta without missing value handling for EDA
- DATA_Texas_District_v2.csv: dropping all missing values with normalization and delta values for Feature Selection and Baseline modeling
- DATA_Texas_District_v3.csv: raw integrated data without normalization, delta. missing value handling for Gradient Boosting experiment

In [ ]:
print('v1{}'.format(df.shape))
df.rename(columns=dict(zip(cols_keep, cols_new)), inplace=True)
df.to_csv('DATA_Texas_District_v1.csv', index=False)

In [ ]:
print('v2{}'.format(df[cols_new].dropna().shape))
df[cols_new].dropna().to_csv('DATA_Texas_District_v2_22.csv', index=False)

In [ ]:
print('v2{}'.format(df[cols_new19].dropna().shape))
df[cols_new19].dropna().to_csv('DATA_Texas_District_v2_19.csv', index=False)

In [ ]:
print('v3{}'.format(df[cols_v3].shape))
df[cols_v3].to_csv('DATA_Texas_District_v3.csv', index=False)